## Выбираем лучшие настройки для тикеров

In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
from bot.test import TestAlgorithm
from app.cache import LocalCache
from bot.vis import TaskProgress
from app.config import RunConfig
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import combinations


# test_configs = [
#     (RunConfig(
#         ticker=t_config.ticker,
#         step_max_cnt=max_shares,
#         step_base_cnt=base_shares,
#         step_lots=1,
# 
#         majority_trade=t_config.majority_trade,
#         pretest_period=pretest_period,
#         pretest_type=t_config.pretest_type,
# 
#         threshold_buy_steps=0,
#         threshold_sell_steps=0,
#         stop_up_p=stop_up_p,
#         stop_down_p=0,
#         start_time=start_time,
# 
#         step_size=t_config.step_size + step_size_shift,
#         step_set_orders_cnt=step_cnt,
#     ))
#     for t_config in [
#         RunConfig.from_repr_string('MTLR+ 2/pre4:0/2 x l1 x 0.4¤'),
#     ]
#     # for max_shares in [3]
#     # for base_shares in [0]
#     # for stop_up_p in [0]
#     # for step_size_shift in [0] # [0, .2, -.2, -.4, .4]
#     # for step_cnt in [2]  # вот тут некоторым удобнее в 2. можно потестить в вариаторе 2 и 3
#     # for pretest_period in [5]  # range(3, 7) # todo
# 
#     for max_shares in [3]
#     for start_time in ['07:00', '07:01', '07:05', '07:10', '07:15', '07:30']
#     for base_shares in [0]
#     for stop_up_p in [0]
#     # for stop_up_p in [0, 0.01]
#     for step_size_shift in [0]
#     # for step_size_shift in [0, .2, -.2, -.4, -.6, -.8]
#     for step_cnt in [2]
#     for pretest_period in [t_config.pretest_period]
#     # for pretest_period in range(3, 7)
# ]

# test_configs = [
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0|'),
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0| L'),
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0| R'),
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0| LR'),
#     # 
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0|'),
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0| L'),
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0| R'),
#     # RunConfig.from_repr_string('MVID+ 2/fan3:0/2 x l6 x 1.6¤ |u0.01 d0.0| LR'),
#     
#     # RunConfig.from_repr_string('MVID+ 2/pre3:0/2 x l6 x 1.6¤ |u0.01 d0.0|'),
#     # RunConfig.from_repr_string('RNFT+ 5/pre5:0/2 x l3 x 0.8¤ |u0.01 d0.0|'),
# ]

# test_configs = [
#     RunConfig.from_repr_string('MVID+ 5/pre3:-5/2 x l1 x 0.8¤'),
#     RunConfig.from_repr_string('MVID+ 4/fan3:0/2 x l1 x 0.2(+x0.2)¤'),
#     RunConfig.from_repr_string('MTLR+ 2/pre4:2/2 x l1 x 1.4¤'),
#     RunConfig.from_repr_string('MTLR+ 2/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('VKCO+ 2/pre4:-2/2 x l1 x 2.2¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('UWGN- 4/fan3:2/2 x l1 x 2.4(+x0.2)¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('SPBE- 18/pre6:9/2 x l1 x 1.6¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('RNFT+ 4/pre4:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('RNFT+ 2/fan6:0/2 x l1 x 0.6(+x0.2)¤'),
# ]

def generate_recombinations(string):
    result = []
    for r in range(len(string) + 1):
        for combo in combinations(string, r):
            result.append("".join(combo))
    return result


test_configs = [
    (RunConfig(
        ticker=t_config.ticker,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        pretest_period=pretest_period,
        pretest_type=t_config.pretest_type,

        stop_up_p=t_config.stop_up_p,

        step_size=t_config.step_size,
        step_size_shift=t_config.step_size_shift,
        step_set_orders_cnt=t_config.step_set_orders_cnt,
        mods=mods,
        # mods=t_config.mods,
    ))
    for t_config in [
        # RunConfig.from_repr_string('RNFT+ 3/fan7:0/2 x l1 x 1.2¤'),
        # RunConfig.from_repr_string('RNFT+ 3/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
        # RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
        # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        
        RunConfig.from_repr_string('MVID+ 5/pre3:-5/2 x l1 x 0.8¤'),
        # RunConfig.from_repr_string('MTLR+ 11/pre5:0/2 x l1 x 0.4¤ F'),
        # RunConfig.from_repr_string('VKCO+ 2/pre4:-2/2 x l1 x 2.2¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('UWGN- 4/fan3:2/2 x l1 x 2.4(+x0.2)¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('SPBE- 18/pre6:9/2 x l1 x 1.6¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('RNFT+ 4/pre4:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MVID+ 4/fan7:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
    ]
    for mods in ['B']
    # for mods in ['', 'M', 'W', 'MW', 'E', 'F', 'B', 'EB', 'FMWEB', 'MWEB']
    # for mods in generate_recombinations('FMWEB')
    for max_shares in [3]
    for base_shares in [0]
    # for stop_up_p in [0]
    # for step_size_shift in [0, .2, .4]
    # for step_cnt in [2]
    for pretest_period in [t_config.pretest_period]
    # for pretest_period in range(3, 8)
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config, use_cache=True)
    return test_alg.test(
        last_test_date='2024-12-31',
        test_days_num=7,
        # test_days_num=366,
        shares_count=0,
        
        try_find_best_config=True,
    )

unique_configs = set(test_configs)

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))


# todo на первом проходе надо 1 ставить, тогда запросы к API будут только 1 раз
with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in unique_configs}

    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()


print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, x['config'].pretest_type, -float(x['profit_p'])), reverse=False)

last_val = ''
for item in sorted_results:
    cur_val = item['config'].ticker
    if cur_val != last_val:
        last_val = cur_val
        print()
    print(item)

Запуск в 21:30
Инициализация....
.
.
.
.
.
.
Закончено в 21:31, количество 1, длительность 0:00:18                                               
cache 481 vals, used 210 times

{'exp': 'MVID pre B', 'profit': 6128, 'profit_p': 6.13, 'profit_p_avg': 0.88, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ B , 'last_conf': MVID+ 4/pre3:4/2 x l139 x 0.6¤ B , 'op': 137}


    {'exp': 'MVID pre M', 'profit': 33885, 'profit_p': 33.88, 'profit_p_avg': 0.09, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ M , 'last_conf': MVID+ 4/pre3:4/2 x l176 x 0.6¤ M , 'op': 5446}
    {'exp': 'MVID pre M', 'profit': 29385, 'profit_p': 29.39, 'profit_p_avg': 0.08, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ M , 'last_conf': MVID+ 4/pre5:4/2 x l170 x 0.6¤ M , 'op': 5244}
    {'exp': 'MVID pre M', 'profit': 29301, 'profit_p': 29.3, 'profit_p_avg': 0.08, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ M , 'last_conf': MVID+ 4/pre6:4/2 x l170 x 0.6¤ M , 'op': 4943}
    {'exp': 'MVID pre M', 'profit': 28784, 'profit_p': 28.78, 'profit_p_avg': 0.08, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ M , 'last_conf': MVID+ 4/pre4:4/2 x l169 x 0.6¤ M , 'op': 5215}
    {'exp': 'MVID pre EMWB', 'profit': 28379, 'profit_p': 28.38, 'profit_p_avg': 0.08, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ EMWB , 'last_conf': MVID+ 4/pre4:0/2 x l179 x 1.0(+x0.2)¤ |u0.01 d0.0| EMWB , 'op': 5996}
    {'exp': 'MVID pre MW', 'profit': 27506, 'profit_p': 27.51, 'profit_p_avg': 0.08, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ MW , 'last_conf': MVID+ 5/pre3:5/2 x l142 x 0.6¤ MW , 'op': 4823}
    {'exp': 'MVID pre MW', 'profit': 27207, 'profit_p': 27.21, 'profit_p_avg': 0.07, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ MW , 'last_conf': MVID+ 4/pre7:4/2 x l177 x 1.0¤ MW , 'op': 4349}
    {'exp': 'MVID pre EFMWB', 'profit': 26074, 'profit_p': 26.07, 'profit_p_avg': 0.07, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ EFMWB , 'last_conf': MVID+ 4/pre3:0/2 x l176 x 1.0(+x0.3)¤ |u0.01 d0.0| EFMWB , 'op': 6222}
    {'exp': 'MVID pre MW', 'profit': 25202, 'profit_p': 25.2, 'profit_p_avg': 0.07, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ MW , 'last_conf': MVID+ 4/pre6:4/2 x l175 x 1.0¤ MW , 'op': 4404}
    {'exp': 'MVID pre EMWB', 'profit': 25102, 'profit_p': 25.1, 'profit_p_avg': 0.07, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ EMWB , 'last_conf': MVID+ 13/pre7:0/2 x l53 x 1.4¤ EMWB , 'op': 5896}
    {'exp': 'MVID pre M', 'profit': 22628, 'profit_p': 22.63, 'profit_p_avg': 0.06, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ M , 'last_conf': MVID+ 4/pre7:4/2 x l161 x 0.6¤ M , 'op': 4648}
    {'exp': 'MVID pre MW', 'profit': 22157, 'profit_p': 22.16, 'profit_p_avg': 0.06, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ MW , 'last_conf': MVID+ 4/pre5:4/2 x l170 x 0.8¤ MW , 'op': 4631}
    {'exp': 'MVID pre EFMWB', 'profit': 21580, 'profit_p': 21.58, 'profit_p_avg': 0.06, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ EFMWB , 'last_conf': MVID+ 13/pre3:0/2 x l52 x 1.4(+x0.3)¤ EFMWB , 'op': 6084}
    {'exp': 'MVID pre B', 'profit': 21214, 'profit_p': 21.21, 'profit_p_avg': 0.06, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ B , 'last_conf': MVID+ 17/pre5:-17/2 x l37 x 1.6(+x0.2)¤ B , 'op': 5892}
    {'exp': 'MVID pre EB', 'profit': 20697, 'profit_p': 20.7, 'profit_p_avg': 0.06, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ EB , 'last_conf': MVID+ 16/pre7:-16/2 x l39 x 1.6(+x0.1)¤ EB , 'op': 5643}
    {'exp': 'MVID pre EMWB', 'profit': 20165, 'profit_p': 20.16, 'profit_p_avg': 0.06, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ EMWB , 'last_conf': MVID+ 7/pre4:0/2 x l96 x 1.4(+x0.2)¤ |u0.01 d0.0| EMWB , 'op': 4968}
    {'exp': 'MVID pre EMWB', 'profit': 19868, 'profit_p': 19.87, 'profit_p_avg': 0.05, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ EMWB , 'last_conf': MVID+ 4/pre4:0/2 x l168 x 1.0(+x0.3)¤ |u0.01 d0.0| EMWB , 'op': 5612}
    {'exp': 'MVID pre EFMWB', 'profit': 19444, 'profit_p': 19.44, 'profit_p_avg': 0.05, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ EFMWB , 'last_conf': MVID+ 8/pre7:0/2 x l83 x 1.4(+x0.3)¤ |u0.01 d0.0| EFMWB , 'op': 5300}
    {'exp': 'MVID pre W', 'profit': 13852, 'profit_p': 13.85, 'profit_p_avg': 0.04, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ W , 'last_conf': MVID+ 4/pre5:4/2 x l158 x 0.6¤ W , 'op': 5358}
    {'exp': 'MVID pre EFMWB', 'profit': 13249, 'profit_p': 13.25, 'profit_p_avg': 0.04, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ EFMWB , 'last_conf': MVID+ 4/pre7:0/2 x l158 x 1.0(+x0.1)¤ |u0.01 d0.0| EFMWB , 'op': 6045}
    {'exp': 'MVID pre E', 'profit': 12666, 'profit_p': 12.67, 'profit_p_avg': 0.03, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ E , 'last_conf': MVID+ 4/pre5:4/2 x l148 x 0.6¤ E , 'op': 5887}
    {'exp': 'MVID pre F', 'profit': 12566, 'profit_p': 12.57, 'profit_p_avg': 0.03, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ F , 'last_conf': MVID+ 4/pre5:4/2 x l148 x 0.6¤ F , 'op': 6223}
    {'exp': 'MVID pre B', 'profit': 11443, 'profit_p': 11.44, 'profit_p_avg': 0.03, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ B , 'last_conf': MVID+ 9/pre4:9/2 x l65 x 0.4(+x0.4)¤ |u0.01 d0.0| B , 'op': 7768}
    {'exp': 'MVID pre ', 'profit': 10987, 'profit_p': 10.99, 'profit_p_avg': 0.03, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ , 'last_conf': MVID+ 4/pre5:4/2 x l146 x 0.6¤ , 'op': 5891}
    {'exp': 'MVID pre EB', 'profit': 10370, 'profit_p': 10.37, 'profit_p_avg': 0.03, 'config': MVID+ 4/pre5:0/2 x l1 x 0.8¤ EB , 'last_conf': MVID+ 4/pre6:4/2 x l145 x 0.6(+x0.4)¤ |u0.01 d0.0| EB , 'op': 6369}
    {'exp': 'MVID pre EMWB', 'profit': 8690, 'profit_p': 8.69, 'profit_p_avg': 0.02, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ EMWB , 'last_conf': MVID+ 4/pre4:0/2 x l152 x 1.0(+x0.1)¤ |u0.01 d0.0| EMWB , 'op': 4867}
    {'exp': 'MVID pre F', 'profit': 7832, 'profit_p': 7.83, 'profit_p_avg': 0.02, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ F , 'last_conf': MVID+ 4/pre7:4/2 x l142 x 0.6¤ F , 'op': 5847}
    {'exp': 'MVID pre W', 'profit': 7431, 'profit_p': 7.43, 'profit_p_avg': 0.02, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ W , 'last_conf': MVID+ 4/pre7:4/2 x l149 x 0.6¤ W , 'op': 5021}
    {'exp': 'MVID pre ', 'profit': 6248, 'profit_p': 6.25, 'profit_p_avg': 0.02, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ , 'last_conf': MVID+ 4/pre7:4/2 x l139 x 0.6¤ , 'op': 5451}
    {'exp': 'MVID pre E', 'profit': 5871, 'profit_p': 5.87, 'profit_p_avg': 0.02, 'config': MVID+ 4/pre7:0/2 x l1 x 0.8¤ E , 'last_conf': MVID+ 4/pre7:4/2 x l139 x 0.6¤ E , 'op': 5443}
    {'exp': 'MVID pre MW', 'profit': 4335, 'profit_p': 4.33, 'profit_p_avg': 0.01, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ MW , 'last_conf': MVID+ 5/pre4:-5/2 x l117 x 0.6¤ MW , 'op': 4269}
    {'exp': 'MVID pre EFMWB', 'profit': 3924, 'profit_p': 3.92, 'profit_p_avg': 0.01, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ EFMWB , 'last_conf': MVID+ 4/pre3:0/2 x l145 x 1.0(+x0.1)¤ |u0.01 d0.0| EFMWB , 'op': 5261}
    {'exp': 'MVID pre W', 'profit': 3828, 'profit_p': 3.83, 'profit_p_avg': 0.01, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ W , 'last_conf': MVID+ 4/pre6:4/2 x l144 x 0.6¤ W , 'op': 4873}
    {'exp': 'MVID pre B', 'profit': 1747, 'profit_p': 1.75, 'profit_p_avg': 0.0, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ B , 'last_conf': MVID+ 4/pre5:4/2 x l134 x 1.0(+x0.1)¤ B , 'op': 5227}
    {'exp': 'MVID pre W', 'profit': 415, 'profit_p': 0.41, 'profit_p_avg': 0.0, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ W , 'last_conf': MVID+ 4/pre3:4/2 x l139 x 0.6¤ W , 'op': 5071}
    {'exp': 'MVID pre B', 'profit': -1012, 'profit_p': -1.01, 'profit_p_avg': -0.0, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ B , 'last_conf': MVID+ 10/pre3:10/2 x l52 x 1.8¤ B , 'op': 3920}
    {'exp': 'MVID pre EB', 'profit': -1348, 'profit_p': -1.35, 'profit_p_avg': -0.0, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ EB , 'last_conf': MVID+ 10/pre4:10/2 x l52 x 1.8¤ EB , 'op': 3905}
    {'exp': 'MVID pre ', 'profit': -2541, 'profit_p': -2.54, 'profit_p_avg': -0.01, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ , 'last_conf': MVID+ 4/pre6:4/2 x l128 x 0.6¤ , 'op': 5369}
    {'exp': 'MVID pre EB', 'profit': -4743, 'profit_p': -4.74, 'profit_p_avg': -0.01, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ EB , 'last_conf': MVID+ 4/pre4:4/2 x l126 x 1.0(+x0.2)¤ EB , 'op': 4944}
    {'exp': 'MVID pre E', 'profit': -5026, 'profit_p': -5.03, 'profit_p_avg': -0.01, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ E , 'last_conf': MVID+ 4/pre6:4/2 x l125 x 0.6¤ E , 'op': 5366}
    {'exp': 'MVID pre F', 'profit': -7745, 'profit_p': -7.75, 'profit_p_avg': -0.02, 'config': MVID+ 4/pre6:0/2 x l1 x 0.8¤ F , 'last_conf': MVID+ 4/pre6:4/2 x l121 x 0.6¤ F , 'op': 5656}
    {'exp': 'MVID pre E', 'profit': -8473, 'profit_p': -8.47, 'profit_p_avg': -0.02, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ E , 'last_conf': MVID+ 4/pre4:4/2 x l120 x 0.6¤ E , 'op': 5819}
    {'exp': 'MVID pre ', 'profit': -8504, 'profit_p': -8.5, 'profit_p_avg': -0.02, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ , 'last_conf': MVID+ 4/pre4:4/2 x l120 x 0.6¤ , 'op': 5698}
    {'exp': 'MVID pre W', 'profit': -8511, 'profit_p': -8.51, 'profit_p_avg': -0.02, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ W , 'last_conf': MVID+ 5/pre4:-5/2 x l103 x 1.6¤ W , 'op': 4852}
    {'exp': 'MVID pre ', 'profit': -9344, 'profit_p': -9.34, 'profit_p_avg': -0.03, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ , 'last_conf': MVID+ 4/pre3:4/2 x l119 x 0.6¤ , 'op': 5853}
    {'exp': 'MVID pre F', 'profit': -10536, 'profit_p': -10.54, 'profit_p_avg': -0.03, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ F , 'last_conf': MVID+ 4/pre4:4/2 x l117 x 0.6¤ F , 'op': 6086}
    {'exp': 'MVID pre F', 'profit': -10714, 'profit_p': -10.71, 'profit_p_avg': -0.03, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ F , 'last_conf': MVID+ 4/pre3:4/2 x l117 x 0.6¤ F , 'op': 6130}
    {'exp': 'MVID pre B', 'profit': -11978, 'profit_p': -11.98, 'profit_p_avg': -0.03, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ B , 'last_conf': MVID+ 4/pre6:4/2 x l115 x 0.6(+x0.1)¤ |u0.01 d0.0| B , 'op': 9720}
    {'exp': 'MVID pre E', 'profit': -12848, 'profit_p': -12.85, 'profit_p_avg': -0.04, 'config': MVID+ 4/pre3:0/2 x l1 x 0.8¤ E , 'last_conf': MVID+ 4/pre3:4/2 x l114 x 0.6¤ E , 'op': 5731}
    {'exp': 'MVID pre EB', 'profit': -14274, 'profit_p': -14.27, 'profit_p_avg': -0.04, 'config': MVID+ 4/pre4:0/2 x l1 x 0.8¤ EB , 'last_conf': MVID+ 4/pre3:4/2 x l112 x 0.6(+x0.2)¤ |u0.01 d0.0| EB , 'op': 9885}

Запуск в 23:59
Закончено в 04:23, количество 8, длительность 4:23:47                                               
cache 485774 vals, used 150836 times

    {'exp': 'RNFT pre EFMWB', 'profit': 334236063, 'profit_p': 334236.06, 'profit_p_avg': 915.72, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 14/pre4:14/2 x l40 x 2.6(+x0.2)¤ EFMWB , 'op': 2347}
    {'exp': 'RNFT pre MW', 'profit': 1691140, 'profit_p': 1691.14, 'profit_p_avg': 4.63, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/pre4:5/2 x l1 x 1.0¤ |u0.01 d0.0| MW , 'op': 4782}
    {'exp': 'RNFT pre W', 'profit': 1007151, 'profit_p': 1007.15, 'profit_p_avg': 2.76, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 7/pre4:7/2 x l95 x 1.4¤ |u0.01 d0.0| W , 'op': 5122}
    {'exp': 'RNFT pre B', 'profit': 16136, 'profit_p': 16.14, 'profit_p_avg': 0.04, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 11/pre7:11/2 x l43 x 2.2(+x0.4)¤ |u0.01 d0.0| B , 'op': 2660}
    {'exp': 'RNFT pre M', 'profit': 12172, 'profit_p': 12.17, 'profit_p_avg': 0.03, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre4:4/2 x l116 x 1.0¤ |u0.01 d0.0| M , 'op': 5945}
    {'exp': 'RNFT pre ', 'profit': -6356, 'profit_p': -6.36, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre4:7/2 x l55 x 1.4¤ |u0.01 d0.0| , 'op': 6108}
    {'exp': 'RNFT pre F', 'profit': -8724, 'profit_p': -8.72, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre4:7/2 x l54 x 1.4¤ |u0.01 d0.0| F , 'op': 6615}
    {'exp': 'RNFT pre E', 'profit': -9479, 'profit_p': -9.48, 'profit_p_avg': -0.03, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre4:7/2 x l53 x 1.4¤ |u0.01 d0.0| E , 'op': 6131}
    
После исправления ошибки с расчетом выходных

    {'exp': 'RNFT pre EFMWB', 'profit': 22908, 'profit_p': 22.91, 'profit_p_avg': 0.06, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 9/pre4:9/2 x l62 x 3.4(+x0.4)¤ |u0.01 d0.0| EFMWB , 'op': 2100}
    {'exp': 'RNFT pre M', 'profit': 12150, 'profit_p': 12.15, 'profit_p_avg': 0.03, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre4:4/2 x l116 x 1.0¤ |u0.01 d0.0| M , 'op': 5928}
    {'exp': 'RNFT pre EB', 'profit': 5944, 'profit_p': 5.94, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 14/pre7:14/2 x l31 x 3.0¤ |u0.01 d0.0| EB , 'op': 3058}
    {'exp': 'RNFT pre MW', 'profit': 883, 'profit_p': 0.88, 'profit_p_avg': 0.0, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/pre4:5/2 x l93 x 0.6¤ |u0.01 d0.0| MW , 'op': 5128}
    {'exp': 'RNFT pre B', 'profit': 452, 'profit_p': 0.45, 'profit_p_avg': 0.0, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 6/pre6:6/2 x l69 x 3.0(+x0.1)¤ |u0.01 d0.0| B , 'op': 2819}
    {'exp': 'RNFT pre ', 'profit': -7896, 'profit_p': -7.9, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre4:7/2 x l54 x 1.4¤ |u0.01 d0.0| , 'op': 6131}
    {'exp': 'RNFT pre W', 'profit': -8866, 'profit_p': -8.87, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 4/pre4:4/2 x l105 x 1.0¤ |u0.01 d0.0| W , 'op': 5585}
    {'exp': 'RNFT pre E', 'profit': -9452, 'profit_p': -9.45, 'profit_p_avg': -0.03, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre4:7/2 x l53 x 1.4¤ |u0.01 d0.0| E , 'op': 6150}
    {'exp': 'RNFT pre F', 'profit': -11334, 'profit_p': -11.33, 'profit_p_avg': -0.03, 'config': RNFT+ 4/pre4:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre4:7/2 x l52 x 1.4¤ |u0.01 d0.0| F , 'op': 6692}
    
Для других pretest_days

    {'exp': 'RNFT pre EB', 'profit': 42003, 'profit_p': 42.0, 'profit_p_avg': 0.11, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 9/pre5:9/2 x l65 x 2.2(+x0.3)¤ EB , 'op': 2565}
    {'exp': 'RNFT pre B', 'profit': 41791, 'profit_p': 41.79, 'profit_p_avg': 0.11, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 9/pre4:9/2 x l65 x 2.2(+x0.1)¤ B , 'op': 2467}
    {'exp': 'RNFT pre EFMWB', 'profit': 35190, 'profit_p': 35.19, 'profit_p_avg': 0.1, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 8/pre7:8/2 x l77 x 1.2(+x0.3)¤ EFMWB , 'op': 5427}
    {'exp': 'RNFT pre EMWB', 'profit': 31858, 'profit_p': 31.86, 'profit_p_avg': 0.09, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 8/pre7:8/2 x l75 x 1.2(+x0.1)¤ EMWB , 'op': 4926}
    {'exp': 'RNFT pre M', 'profit': 29167, 'profit_p': 29.17, 'profit_p_avg': 0.08, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 5/pre7:5/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'op': 5370}
    {'exp': 'RNFT pre MW', 'profit': 22447, 'profit_p': 22.45, 'profit_p_avg': 0.06, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/pre7:5/2 x l113 x 0.8¤ |u0.01 d0.0| MW , 'op': 5455}
    {'exp': 'RNFT pre EMWB', 'profit': 21597, 'profit_p': 21.6, 'profit_p_avg': 0.06, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 8/pre7:8/2 x l69 x 2.2(+x0.2)¤ EMWB , 'op': 2639}
    {'exp': 'RNFT pre MW', 'profit': 18787, 'profit_p': 18.79, 'profit_p_avg': 0.05, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/pre6:5/2 x l109 x 0.8¤ |u0.01 d0.0| MW , 'op': 5513}
    {'exp': 'RNFT pre M', 'profit': 17709, 'profit_p': 17.71, 'profit_p_avg': 0.05, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre6:4/2 x l122 x 1.0¤ |u0.01 d0.0| M , 'op': 6383}
    {'exp': 'RNFT pre EFMWB', 'profit': 17573, 'profit_p': 17.57, 'profit_p_avg': 0.05, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 7/pre4:7/2 x l76 x 2.2(+x0.4)¤ EFMWB , 'op': 2464}
    {'exp': 'RNFT pre EFMWB', 'profit': 16212, 'profit_p': 16.21, 'profit_p_avg': 0.04, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 13/pre5:13/2 x l40 x 0.6(+x0.1)¤ EFMWB , 'op': 7140}
    {'exp': 'RNFT pre EMWB', 'profit': 15823, 'profit_p': 15.82, 'profit_p_avg': 0.04, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 13/pre5:13/2 x l40 x 0.6¤ EMWB , 'op': 6856}
    {'exp': 'RNFT pre B', 'profit': 11171, 'profit_p': 11.17, 'profit_p_avg': 0.03, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 6/pre4:6/2 x l77 x 3.4(+x0.3)¤ B , 'op': 2651}
    {'exp': 'RNFT pre EB', 'profit': 9724, 'profit_p': 9.72, 'profit_p_avg': 0.03, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 14/pre5:14/2 x l32 x 3.2(+x0.3)¤ EB , 'op': 2514}
    {'exp': 'RNFT pre W', 'profit': 6676, 'profit_p': 6.68, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 4/pre6:4/2 x l123 x 1.0¤ |u0.01 d0.0| W , 'op': 6564}
    {'exp': 'RNFT pre EB', 'profit': 6650, 'profit_p': 6.65, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 14/pre3:14/2 x l31 x 1.2(+x0.2)¤ |u0.01 d0.0| EB , 'op': 6318}
    {'exp': 'RNFT pre M', 'profit': 5740, 'profit_p': 5.74, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre5:4/2 x l113 x 1.0¤ |u0.01 d0.0| M , 'op': 5876}
    {'exp': 'RNFT pre E', 'profit': 5289, 'profit_p': 5.29, 'profit_p_avg': 0.01, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/pre7:6/2 x l73 x 1.2¤ |u0.01 d0.0| E , 'op': 6802}
    {'exp': 'RNFT pre F', 'profit': 4955, 'profit_p': 4.96, 'profit_p_avg': 0.01, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre7:7/2 x l62 x 1.4¤ |u0.01 d0.0| F , 'op': 6925}
    {'exp': 'RNFT pre B', 'profit': 3003, 'profit_p': 3.0, 'profit_p_avg': 0.01, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 11/pre3:11/2 x l38 x 2.2(+x0.1)¤ B , 'op': 4018}
    {'exp': 'RNFT pre ', 'profit': 412, 'profit_p': 0.41, 'profit_p_avg': 0.0, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre7:7/2 x l59 x 1.4¤ |u0.01 d0.0| , 'op': 6751}
    {'exp': 'RNFT pre ', 'profit': -2955, 'profit_p': -2.96, 'profit_p_avg': -0.01, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre6:7/2 x l67 x 1.4¤ |u0.01 d0.0| , 'op': 7147}
    {'exp': 'RNFT pre MW', 'profit': -3624, 'profit_p': -3.62, 'profit_p_avg': -0.01, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/pre5:5/2 x l89 x 0.8¤ |u0.01 d0.0| MW , 'op': 4667}
    {'exp': 'RNFT pre E', 'profit': -4287, 'profit_p': -4.29, 'profit_p_avg': -0.01, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre6:7/2 x l67 x 1.4¤ |u0.01 d0.0| E , 'op': 6969}
    {'exp': 'RNFT pre F', 'profit': -4638, 'profit_p': -4.64, 'profit_p_avg': -0.01, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre6:7/2 x l67 x 1.4¤ |u0.01 d0.0| F , 'op': 7009}
    {'exp': 'RNFT pre W', 'profit': -7138, 'profit_p': -7.14, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 4/pre7:4/2 x l107 x 1.0¤ |u0.01 d0.0| W , 'op': 6494}
    {'exp': 'RNFT pre F', 'profit': -8100, 'profit_p': -8.1, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre5:7/2 x l54 x 1.4¤ |u0.01 d0.0| F , 'op': 7569}
    {'exp': 'RNFT pre ', 'profit': -9735, 'profit_p': -9.73, 'profit_p_avg': -0.03, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre5:7/2 x l66 x 1.4¤ |u0.01 d0.0| , 'op': 7424}
    {'exp': 'RNFT pre E', 'profit': -9867, 'profit_p': -9.87, 'profit_p_avg': -0.03, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre5:7/2 x l66 x 1.4¤ |u0.01 d0.0| E , 'op': 7182}
    {'exp': 'RNFT pre W', 'profit': -14366, 'profit_p': -14.37, 'profit_p_avg': -0.04, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 4/pre5:4/2 x l99 x 1.0¤ |u0.01 d0.0| W , 'op': 5889}

Запуск в 04:31
Закончено в 07:25, количество 27, длительность 1 day, 2:53:57                                       
cache 2541412 vals, used 1048664 times

    {'exp': 'RNFT pre EFMWB', 'profit': 71589422800, 'profit_p': 71589422.8, 'profit_p_avg': 195599.52, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 17/pre3:17/2 x l32 x 2.2(+x0.4)¤ EFMWB , 'op': 2833}
    {'exp': 'RNFT pre EFMWB', 'profit': 1574552128, 'profit_p': 1574552.13, 'profit_p_avg': 4302.05, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 13/pre6:13/2 x l47 x 2.2¤ EFMWB , 'op': 2164}
    {'exp': 'RNFT pre EFMWB', 'profit': 553889690, 'profit_p': 553889.69, 'profit_p_avg': 1513.36, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 14/pre5:14/2 x l39 x 1.8¤ |u0.01 d0.0| EFMWB , 'op': 2783}
    {'exp': 'RNFT pre W', 'profit': 195862625, 'profit_p': 195862.62, 'profit_p_avg': 535.14, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 6/pre7:6/2 x l95 x 1.2¤ |u0.01 d0.0| W , 'op': 4635}
    {'exp': 'RNFT pre MW', 'profit': 22258118, 'profit_p': 22258.12, 'profit_p_avg': 60.81, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 4/pre7:4/2 x l1 x 1.0¤ |u0.01 d0.0| MW , 'op': 4395}
    {'exp': 'RNFT pre MW', 'profit': 3375966, 'profit_p': 3375.97, 'profit_p_avg': 9.22, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 4/pre6:4/2 x l1 x 1.0¤ |u0.01 d0.0| MW , 'op': 4654}
    {'exp': 'RNFT pre W', 'profit': 3268386, 'profit_p': 3268.39, 'profit_p_avg': 8.93, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 7/pre6:7/2 x l95 x 1.4¤ |u0.01 d0.0| W , 'op': 5406}
    {'exp': 'RNFT pre W', 'profit': 1363797, 'profit_p': 1363.8, 'profit_p_avg': 3.73, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 7/pre5:7/2 x l94 x 1.4¤ |u0.01 d0.0| W , 'op': 5628}
    {'exp': 'RNFT pre MW', 'profit': 727247, 'profit_p': 727.25, 'profit_p_avg': 1.99, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/pre5:5/2 x l117 x 1.0¤ |u0.01 d0.0| MW , 'op': 5103}
    {'exp': 'RNFT pre EB', 'profit': 43016, 'profit_p': 43.02, 'profit_p_avg': 0.12, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 7/pre5:7/2 x l85 x 3.2(+x0.4)¤ EB , 'op': 2180}
    {'exp': 'RNFT pre B', 'profit': 41597, 'profit_p': 41.6, 'profit_p_avg': 0.11, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 10/pre7:10/2 x l58 x 3.2(+x0.4)¤ B , 'op': 2127}
    {'exp': 'RNFT pre M', 'profit': 32355, 'profit_p': 32.35, 'profit_p_avg': 0.09, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre7:4/2 x l137 x 1.0¤ |u0.01 d0.0| M , 'op': 5512}
    {'exp': 'RNFT pre M', 'profit': 17353, 'profit_p': 17.35, 'profit_p_avg': 0.05, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre6:4/2 x l122 x 1.0¤ |u0.01 d0.0| M , 'op': 6305}
    {'exp': 'RNFT pre B', 'profit': 11502, 'profit_p': 11.5, 'profit_p_avg': 0.03, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 9/pre4:9/2 x l51 x 2.2(+x0.4)¤ B , 'op': 4298}
    {'exp': 'RNFT pre F', 'profit': 11049, 'profit_p': 11.05, 'profit_p_avg': 0.03, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre7:7/2 x l66 x 1.4¤ |u0.01 d0.0| F , 'op': 6645}
    {'exp': 'RNFT pre EB', 'profit': 6644, 'profit_p': 6.64, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 14/pre3:14/2 x l31 x 1.2(+x0.4)¤ |u0.01 d0.0| EB , 'op': 6522}
    {'exp': 'RNFT pre EB', 'profit': 5632, 'profit_p': 5.63, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 10/pre4:10/2 x l43 x 2.2(+x0.1)¤ EB , 'op': 3312}
    {'exp': 'RNFT pre E', 'profit': 5588, 'profit_p': 5.59, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre7:7/2 x l62 x 1.4¤ |u0.01 d0.0| E , 'op': 6537}
    {'exp': 'RNFT pre ', 'profit': 5580, 'profit_p': 5.58, 'profit_p_avg': 0.02, 'config': RNFT+ 4/pre7:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre7:7/2 x l62 x 1.4¤ |u0.01 d0.0| , 'op': 6537}
    {'exp': 'RNFT pre M', 'profit': 3415, 'profit_p': 3.42, 'profit_p_avg': 0.01, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 4/pre5:4/2 x l107 x 1.0¤ |u0.01 d0.0| M , 'op': 6397}
    {'exp': 'RNFT pre ', 'profit': 1717, 'profit_p': 1.72, 'profit_p_avg': 0.0, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre6:7/2 x l60 x 1.4¤ |u0.01 d0.0| , 'op': 7081}
    {'exp': 'RNFT pre F', 'profit': -1282, 'profit_p': -1.28, 'profit_p_avg': -0.0, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre6:7/2 x l58 x 1.4¤ |u0.01 d0.0| F , 'op': 7221}
    {'exp': 'RNFT pre B', 'profit': -2977, 'profit_p': -2.98, 'profit_p_avg': -0.01, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 11/pre4:11/2 x l36 x 1.2(+x0.2)¤ |u0.01 d0.0| B , 'op': 6520}
    {'exp': 'RNFT pre E', 'profit': -4300, 'profit_p': -4.3, 'profit_p_avg': -0.01, 'config': RNFT+ 4/pre6:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre6:7/2 x l56 x 1.4¤ |u0.01 d0.0| E , 'op': 7014}
    {'exp': 'RNFT pre F', 'profit': -8248, 'profit_p': -8.25, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 7/pre5:7/2 x l54 x 1.4¤ |u0.01 d0.0| F , 'op': 7605}
    {'exp': 'RNFT pre ', 'profit': -8742, 'profit_p': -8.74, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 7/pre5:7/2 x l54 x 1.4¤ |u0.01 d0.0| , 'op': 7192}
    {'exp': 'RNFT pre E', 'profit': -9090, 'profit_p': -9.09, 'profit_p_avg': -0.02, 'config': RNFT+ 4/pre5:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 7/pre5:7/2 x l54 x 1.4¤ |u0.01 d0.0| E , 'op': 7178}

Запуск в 14:57
Закончено в 18:12, количество 80, длительность 3:15:26                                              
cache 569615 vals, used 423583 times

    {'exp': 'RNFT fan E', 'profit': -598, 'profit_p': -0.6, 'profit_p_avg': -0.0, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ E , 'last_conf': RNFT+ 6/fan7:0/2 x l73 x 1.4¤ E , 'op': 5024}
    {'exp': 'RNFT fan E', 'profit': -886, 'profit_p': -0.89, 'profit_p_avg': -0.0, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/fan7:0/2 x l73 x 1.4¤ |u0.01 d0.0| E , 'op': 4627}
    {'exp': 'RNFT fan EFMWB', 'profit': -2726, 'profit_p': -2.73, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 4/fan6:0/2 x l89 x 3.8(+x0.4)¤ EFMWB , 'op': 1230}
    {'exp': 'RNFT fan EFMWB', 'profit': -2910, 'profit_p': -2.91, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 4/fan4:0/2 x l116 x 4.6(+x0.3)¤ EFMWB , 'op': 1539}
    {'exp': 'RNFT fan EFMWB', 'profit': -3313, 'profit_p': -3.31, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ EFMWB , 'last_conf': RNFT+ 4/fan4:0/2 x l116 x 4.6(+x0.3)¤ |u0.01 d0.0| EFMWB , 'op': 1517}
    {'exp': 'RNFT fan B', 'profit': -3339, 'profit_p': -3.34, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 6/fan6:0/2 x l73 x 1.4¤ B , 'op': 3237}
    {'exp': 'RNFT fan ', 'profit': -3442, 'profit_p': -3.44, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| , 'last_conf': RNFT+ 5/fan7:0/2 x l88 x 1.6¤ |u0.01 d0.0| , 'op': 2613}
    {'exp': 'RNFT fan EFMWB', 'profit': -3677, 'profit_p': -3.68, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 4/fan4:0/2 x l117 x 2.8(+x0.3)¤ EFMWB , 'op': 1554}
    {'exp': 'RNFT fan EFMWB', 'profit': -3810, 'profit_p': -3.81, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ EFMWB , 'last_conf': RNFT+ 4/fan4:0/2 x l116 x 2.8(+x0.3)¤ |u0.01 d0.0| EFMWB , 'op': 1544}
    {'exp': 'RNFT fan W', 'profit': -4196, 'profit_p': -4.2, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 5/fan7:0/2 x l77 x 1.6¤ |u0.01 d0.0| W , 'op': 2610}
    {'exp': 'RNFT fan B', 'profit': -4504, 'profit_p': -4.5, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ B , 'last_conf': RNFT+ 13/fan7:0/2 x l34 x 0.6¤ B , 'op': 2850}
    {'exp': 'RNFT fan B', 'profit': -4679, 'profit_p': -4.68, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ B , 'last_conf': RNFT+ 4/fan3:0/2 x l111 x 2.4(+x0.1)¤ B , 'op': 3203}
    {'exp': 'RNFT fan B', 'profit': -4987, 'profit_p': -4.99, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 4/fan5:0/2 x l111 x 2.4(+x0.1)¤ B , 'op': 3203}
    {'exp': 'RNFT fan B', 'profit': -5019, 'profit_p': -5.02, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ B , 'last_conf': RNFT+ 4/fan6:0/2 x l98 x 2.4(+x0.1)¤ B , 'op': 3465}
    {'exp': 'RNFT fan ', 'profit': -5418, 'profit_p': -5.42, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 5/fan7:0/2 x l88 x 1.6¤ , 'op': 2610}
    {'exp': 'RNFT fan B', 'profit': -6156, 'profit_p': -6.16, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 13/fan7:0/2 x l27 x 0.6¤ B , 'op': 3608}
    {'exp': 'RNFT fan M', 'profit': -6282, 'profit_p': -6.28, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ M , 'last_conf': RNFT+ 6/fan7:0/2 x l73 x 1.4¤ M , 'op': 3056}
    {'exp': 'RNFT fan MW', 'profit': -6458, 'profit_p': -6.46, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/fan7:0/2 x l77 x 1.8¤ |u0.01 d0.0| MW , 'op': 2880}
    {'exp': 'RNFT fan EFMWB', 'profit': -6542, 'profit_p': -6.54, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ EFMWB , 'last_conf': RNFT+ 5/fan4:0/2 x l92 x 1.6(+x0.4)¤ EFMWB , 'op': 2131}
    {'exp': 'RNFT fan EFMWB', 'profit': -6676, 'profit_p': -6.68, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 4/fan6:0/2 x l115 x 4.0¤ |u0.01 d0.0| EFMWB , 'op': 1028}
    {'exp': 'RNFT fan MW', 'profit': -6679, 'profit_p': -6.68, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ MW , 'last_conf': RNFT+ 8/fan7:0/2 x l1 x 1.0¤ MW , 'op': 3066}
    {'exp': 'RNFT fan M', 'profit': -6790, 'profit_p': -6.79, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 6/fan7:0/2 x l73 x 1.4¤ |u0.01 d0.0| M , 'op': 3089}
    {'exp': 'RNFT fan W', 'profit': -6964, 'profit_p': -6.96, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ W , 'last_conf': RNFT+ 5/fan7:0/2 x l77 x 1.6¤ W , 'op': 2530}
    {'exp': 'RNFT fan B', 'profit': -7717, 'profit_p': -7.72, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 6/fan5:0/2 x l74 x 1.4¤ B , 'op': 3763}
    {'exp': 'RNFT fan E', 'profit': -7731, 'profit_p': -7.73, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/fan6:0/2 x l73 x 1.4¤ |u0.01 d0.0| E , 'op': 4903}
    {'exp': 'RNFT fan E', 'profit': -8054, 'profit_p': -8.05, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ E , 'last_conf': RNFT+ 6/fan6:0/2 x l73 x 1.4¤ E , 'op': 4877}
    {'exp': 'RNFT fan M', 'profit': -8405, 'profit_p': -8.4, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ M , 'last_conf': RNFT+ 6/fan6:0/2 x l73 x 1.4¤ M , 'op': 2773}
    {'exp': 'RNFT fan MW', 'profit': -9131, 'profit_p': -9.13, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ MW , 'last_conf': RNFT+ 5/fan6:0/2 x l77 x 1.8¤ MW , 'op': 2784}
    {'exp': 'RNFT fan M', 'profit': -9720, 'profit_p': -9.72, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 6/fan6:0/2 x l73 x 1.4¤ |u0.01 d0.0| M , 'op': 2863}
    {'exp': 'RNFT fan EFMWB', 'profit': -9823, 'profit_p': -9.82, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 6/fan6:0/2 x l65 x 1.4¤ EFMWB , 'op': 4294}
    {'exp': 'RNFT fan MW', 'profit': -10369, 'profit_p': -10.37, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 5/fan6:0/2 x l77 x 1.8¤ |u0.01 d0.0| MW , 'op': 2777}
    {'exp': 'RNFT fan EFMWB', 'profit': -10533, 'profit_p': -10.53, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ EFMWB , 'last_conf': RNFT+ 6/fan3:0/2 x l76 x 1.4¤ EFMWB , 'op': 4244}
    {'exp': 'RNFT fan W', 'profit': -10725, 'profit_p': -10.72, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ W , 'last_conf': RNFT+ 8/fan6:0/2 x l46 x 1.0¤ W , 'op': 2974}
    {'exp': 'RNFT fan EFMWB', 'profit': -11191, 'profit_p': -11.19, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ EFMWB , 'last_conf': RNFT+ 6/fan3:0/2 x l51 x 1.4¤ |u0.01 d0.0| EFMWB , 'op': 4012}
    {'exp': 'RNFT fan ', 'profit': -11285, 'profit_p': -11.29, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 6/fan6:0/2 x l73 x 1.4¤ , 'op': 2929}
    {'exp': 'RNFT fan W', 'profit': -11337, 'profit_p': -11.34, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 8/fan6:0/2 x l46 x 1.0¤ |u0.01 d0.0| W , 'op': 2949}
    {'exp': 'RNFT fan ', 'profit': -11466, 'profit_p': -11.47, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| , 'last_conf': RNFT+ 6/fan5:0/2 x l73 x 1.4¤ |u0.01 d0.0| , 'op': 2356}
    {'exp': 'RNFT fan MW', 'profit': -11576, 'profit_p': -11.58, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ MW , 'last_conf': RNFT+ 7/fan3:0/2 x l1 x 1.2¤ MW , 'op': 2446}
    {'exp': 'RNFT fan MW', 'profit': -11679, 'profit_p': -11.68, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 4/fan4:0/2 x l92 x 2.0¤ |u0.01 d0.0| MW , 'op': 2057}
    {'exp': 'RNFT fan MW', 'profit': -11776, 'profit_p': -11.78, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ MW , 'last_conf': RNFT+ 4/fan4:0/2 x l92 x 2.0¤ MW , 'op': 2091}
    {'exp': 'RNFT fan B', 'profit': -11821, 'profit_p': -11.82, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ B , 'last_conf': RNFT+ 6/fan6:0/2 x l74 x 1.4¤ B , 'op': 3533}
    {'exp': 'RNFT fan ', 'profit': -11818, 'profit_p': -11.82, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 6/fan5:0/2 x l73 x 1.4¤ , 'op': 2493}
    {'exp': 'RNFT fan E', 'profit': -11906, 'profit_p': -11.91, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/fan4:0/2 x l74 x 1.4¤ |u0.01 d0.0| E , 'op': 3509}
    {'exp': 'RNFT fan ', 'profit': -12338, 'profit_p': -12.34, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| , 'last_conf': RNFT+ 5/fan6:0/2 x l88 x 1.6¤ |u0.01 d0.0| , 'op': 2963}
    {'exp': 'RNFT fan B', 'profit': -12933, 'profit_p': -12.93, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ B , 'last_conf': RNFT+ 4/fan5:0/2 x l111 x 2.4(+x0.1)¤ B , 'op': 3564}
    {'exp': 'RNFT fan B', 'profit': -13122, 'profit_p': -13.12, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 4/fan5:0/2 x l111 x 2.4(+x0.1)¤ |u0.01 d0.0| B , 'op': 3637}
    {'exp': 'RNFT fan MW', 'profit': -13143, 'profit_p': -13.14, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 7/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| MW , 'op': 2497}
    {'exp': 'RNFT fan E', 'profit': -13638, 'profit_p': -13.64, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ E , 'last_conf': RNFT+ 6/fan4:0/2 x l74 x 1.4¤ E , 'op': 3524}
    {'exp': 'RNFT fan W', 'profit': -13875, 'profit_p': -13.88, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 8/fan5:0/2 x l57 x 1.0¤ |u0.01 d0.0| W , 'op': 2459}
    {'exp': 'RNFT fan W', 'profit': -14187, 'profit_p': -14.19, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ W , 'last_conf': RNFT+ 8/fan5:0/2 x l57 x 1.0¤ W , 'op': 2502}
    {'exp': 'RNFT fan ', 'profit': -14725, 'profit_p': -14.72, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| , 'last_conf': RNFT+ 6/fan4:0/2 x l73 x 1.4¤ |u0.01 d0.0| , 'op': 2398}
    {'exp': 'RNFT fan E', 'profit': -14776, 'profit_p': -14.78, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/fan5:0/2 x l74 x 1.4¤ |u0.01 d0.0| E , 'op': 4130}
    {'exp': 'RNFT fan W', 'profit': -14938, 'profit_p': -14.94, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 10/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'op': 2176}
    {'exp': 'RNFT fan W', 'profit': -15014, 'profit_p': -15.01, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ W , 'last_conf': RNFT+ 10/fan4:0/2 x l1 x 0.8¤ W , 'op': 2149}
    {'exp': 'RNFT fan ', 'profit': -15063, 'profit_p': -15.06, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 6/fan4:0/2 x l73 x 1.4¤ , 'op': 2306}
    {'exp': 'RNFT fan E', 'profit': -15889, 'profit_p': -15.89, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ E , 'last_conf': RNFT+ 6/fan5:0/2 x l74 x 1.4¤ E , 'op': 4150}
    {'exp': 'RNFT fan M', 'profit': -16533, 'profit_p': -16.53, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ M , 'last_conf': RNFT+ 6/fan3:0/2 x l57 x 1.4¤ M , 'op': 2665}
    {'exp': 'RNFT fan M', 'profit': -16652, 'profit_p': -16.65, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 7/fan4:0/2 x l63 x 1.2¤ |u0.01 d0.0| M , 'op': 2306}
    {'exp': 'RNFT fan M', 'profit': -16871, 'profit_p': -16.87, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ M , 'last_conf': RNFT+ 7/fan4:0/2 x l63 x 1.2¤ M , 'op': 2339}
    {'exp': 'RNFT fan MW', 'profit': -17728, 'profit_p': -17.73, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ MW , 'last_conf': RNFT+ 4/fan5:0/2 x l92 x 2.0¤ MW , 'op': 2497}
    {'exp': 'RNFT fan MW', 'profit': -17865, 'profit_p': -17.86, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 4/fan5:0/2 x l92 x 2.0¤ |u0.01 d0.0| MW , 'op': 2443}
    {'exp': 'RNFT fan M', 'profit': -17914, 'profit_p': -17.91, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 7/fan5:0/2 x l63 x 1.2¤ |u0.01 d0.0| M , 'op': 2424}
    {'exp': 'RNFT fan M', 'profit': -18278, 'profit_p': -18.28, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ M , 'last_conf': RNFT+ 7/fan5:0/2 x l63 x 1.2¤ M , 'op': 2512}
    {'exp': 'RNFT fan M', 'profit': -18458, 'profit_p': -18.46, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 6/fan3:0/2 x l56 x 1.4¤ |u0.01 d0.0| M , 'op': 2839}
    {'exp': 'RNFT fan ', 'profit': -19191, 'profit_p': -19.19, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 6/fan3:0/2 x l74 x 1.4¤ , 'op': 2724}
    {'exp': 'RNFT fan ', 'profit': -19624, 'profit_p': -19.62, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| , 'last_conf': RNFT+ 6/fan3:0/2 x l74 x 1.4¤ |u0.01 d0.0| , 'op': 2812}
    {'exp': 'RNFT fan W', 'profit': -20232, 'profit_p': -20.23, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 13/fan3:0/2 x l57 x 0.6¤ |u0.01 d0.0| W , 'op': 2646}
    {'exp': 'RNFT fan W', 'profit': -20330, 'profit_p': -20.33, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ W , 'last_conf': RNFT+ 13/fan3:0/2 x l57 x 0.6¤ W , 'op': 2601}
    {'exp': 'RNFT fan F', 'profit': -21288, 'profit_p': -21.29, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 5/fan7:0/2 x l65 x 1.6¤ |u0.01 d0.0| F , 'op': 5406}
    {'exp': 'RNFT fan F', 'profit': -22111, 'profit_p': -22.11, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ F , 'last_conf': RNFT+ 5/fan7:0/2 x l64 x 1.6¤ F , 'op': 5271}
    {'exp': 'RNFT fan E', 'profit': -22326, 'profit_p': -22.33, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/fan3:0/2 x l74 x 1.4¤ |u0.01 d0.0| E , 'op': 3492}
    {'exp': 'RNFT fan E', 'profit': -22827, 'profit_p': -22.83, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ E , 'last_conf': RNFT+ 6/fan3:0/2 x l74 x 1.4¤ E , 'op': 3460}
    {'exp': 'RNFT fan F', 'profit': -23858, 'profit_p': -23.86, 'profit_p_avg': -0.07, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 5/fan6:0/2 x l63 x 1.6¤ |u0.01 d0.0| F , 'op': 5523}
    {'exp': 'RNFT fan F', 'profit': -25034, 'profit_p': -25.03, 'profit_p_avg': -0.07, 'config': RNFT+ 4/fan6:0/2 x l1 x 1.2¤ F , 'last_conf': RNFT+ 6/fan6:0/2 x l51 x 1.4¤ F , 'op': 5463}
    {'exp': 'RNFT fan F', 'profit': -26423, 'profit_p': -26.42, 'profit_p_avg': -0.07, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 6/fan5:0/2 x l50 x 1.4¤ |u0.01 d0.0| F , 'op': 5347}
    {'exp': 'RNFT fan F', 'profit': -28562, 'profit_p': -28.56, 'profit_p_avg': -0.08, 'config': RNFT+ 4/fan5:0/2 x l1 x 1.2¤ F , 'last_conf': RNFT+ 6/fan5:0/2 x l49 x 1.4¤ F , 'op': 5358}
    {'exp': 'RNFT fan F', 'profit': -28945, 'profit_p': -28.95, 'profit_p_avg': -0.08, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 6/fan4:0/2 x l49 x 1.4¤ |u0.01 d0.0| F , 'op': 5093}
    {'exp': 'RNFT fan F', 'profit': -29979, 'profit_p': -29.98, 'profit_p_avg': -0.08, 'config': RNFT+ 4/fan4:0/2 x l1 x 1.2¤ F , 'last_conf': RNFT+ 6/fan4:0/2 x l48 x 1.4¤ F , 'op': 4842}
    {'exp': 'RNFT fan F', 'profit': -32324, 'profit_p': -32.32, 'profit_p_avg': -0.09, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 6/fan3:0/2 x l46 x 1.4¤ |u0.01 d0.0| F , 'op': 5763}
    {'exp': 'RNFT fan F', 'profit': -32462, 'profit_p': -32.46, 'profit_p_avg': -0.09, 'config': RNFT+ 4/fan3:0/2 x l1 x 1.2¤ F , 'last_conf': RNFT+ 6/fan3:0/2 x l46 x 1.4¤ F , 'op': 5798}

FAN

    {'exp': 'RNFT fan EB', 'profit': -1191, 'profit_p': -1.19, 'profit_p_avg': -0.0, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 5/fan6:0/2 x l88 x 1.6(+x0.1)¤ EB , 'op': 4673}
    {'exp': 'RNFT fan EB', 'profit': -2299, 'profit_p': -2.3, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 6/fan5:0/2 x l73 x 1.4(+x0.1)¤ EB , 'op': 5005}
    {'exp': 'RNFT fan B', 'profit': -3071, 'profit_p': -3.07, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 4/fan4:0/2 x l111 x 2.4(+x0.1)¤ B , 'op': 3253}
    {'exp': 'RNFT fan E', 'profit': -3875, 'profit_p': -3.88, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 8/fan7:0/2 x l55 x 1.0¤ |u0.01 d0.0| E , 'op': 6727}
    {'exp': 'RNFT fan MW', 'profit': -4104, 'profit_p': -4.1, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 6/fan7:0/2 x l74 x 1.6¤ |u0.01 d0.0| MW , 'op': 3960}
    {'exp': 'RNFT fan M', 'profit': -4348, 'profit_p': -4.35, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 5/fan7:0/2 x l88 x 1.8¤ |u0.01 d0.0| M , 'op': 4242}
    {'exp': 'RNFT fan EMWB', 'profit': -4439, 'profit_p': -4.44, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 6/fan6:0/2 x l82 x 1.8(+x0.3)¤ |u0.01 d0.0| EMWB , 'op': 1243}
    {'exp': 'RNFT fan EB', 'profit': -4667, 'profit_p': -4.67, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 6/fan3:0/2 x l73 x 1.4(+x0.1)¤ EB , 'op': 3261}
    {'exp': 'RNFT fan EMWB', 'profit': -4822, 'profit_p': -4.82, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 4/fan6:0/2 x l110 x 2.4(+x0.4)¤ |u0.01 d0.0| EMWB , 'op': 1475}
    {'exp': 'RNFT fan EMWB', 'profit': -4890, 'profit_p': -4.89, 'profit_p_avg': -0.01, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 10/fan7:0/2 x l49 x 1.0¤ EMWB , 'op': 3429}
    {'exp': 'RNFT fan W', 'profit': -5759, 'profit_p': -5.76, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 6/fan7:0/2 x l72 x 1.6¤ |u0.01 d0.0| W , 'op': 4099}
    {'exp': 'RNFT fan ', 'profit': -5969, 'profit_p': -5.97, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 8/fan7:0/2 x l55 x 1.0¤ |u0.01 d0.0| , 'op': 4516}
    {'exp': 'RNFT fan EMWB', 'profit': -6340, 'profit_p': -6.34, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 8/fan7:0/2 x l54 x 1.2¤ EMWB , 'op': 2239}
    {'exp': 'RNFT fan EB', 'profit': -7507, 'profit_p': -7.51, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 4/fan5:0/2 x l111 x 2.8¤ |u0.01 d0.0| EB , 'op': 1828}
    {'exp': 'RNFT fan EFMWB', 'profit': -8020, 'profit_p': -8.02, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 4/fan4:0/2 x l106 x 2.4(+x0.4)¤ EFMWB , 'op': 3678}
    {'exp': 'RNFT fan MW', 'profit': -8917, 'profit_p': -8.92, 'profit_p_avg': -0.02, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 6/fan6:0/2 x l70 x 1.8¤ |u0.01 d0.0| MW , 'op': 4032}
    {'exp': 'RNFT fan EMWB', 'profit': -9148, 'profit_p': -9.15, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': RNFT+ 6/fan4:0/2 x l70 x 1.4¤ |u0.01 d0.0| EMWB , 'op': 4010}
    {'exp': 'RNFT fan EFMWB', 'profit': -9357, 'profit_p': -9.36, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 8/fan6:0/2 x l52 x 1.2¤ EFMWB , 'op': 8843}
    {'exp': 'RNFT fan M', 'profit': -9868, 'profit_p': -9.87, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 10/fan6:0/2 x l44 x 0.8¤ |u0.01 d0.0| M , 'op': 4414}
    {'exp': 'RNFT fan B', 'profit': -10141, 'profit_p': -10.14, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 5/fan7:0/2 x l88 x 1.6¤ B , 'op': 3498}
    {'exp': 'RNFT fan E', 'profit': -11249, 'profit_p': -11.25, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 8/fan6:0/2 x l55 x 1.0¤ |u0.01 d0.0| E , 'op': 6372}
    {'exp': 'RNFT fan B', 'profit': -11246, 'profit_p': -11.25, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 4/fan4:0/2 x l111 x 2.4(+x0.1)¤ B , 'op': 3192}
    {'exp': 'RNFT fan EFMWB', 'profit': -11446, 'profit_p': -11.45, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 6/fan6:0/2 x l68 x 1.8(+x0.3)¤ EFMWB , 'op': 3758}
    {'exp': 'RNFT fan EB', 'profit': -11567, 'profit_p': -11.57, 'profit_p_avg': -0.03, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': RNFT+ 6/fan4:0/2 x l74 x 1.4(+x0.1)¤ |u0.01 d0.0| EB , 'op': 1385}
    {'exp': 'RNFT fan B', 'profit': -13024, 'profit_p': -13.02, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 6/fan6:0/2 x l74 x 1.4¤ B , 'op': 4328}
    {'exp': 'RNFT fan EFMWB', 'profit': -13148, 'profit_p': -13.15, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 8/fan5:0/2 x l50 x 1.2¤ EFMWB , 'op': 7109}
    {'exp': 'RNFT fan B', 'profit': -13151, 'profit_p': -13.15, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': RNFT+ 13/fan3:0/2 x l34 x 0.6¤ B , 'op': 4328}
    {'exp': 'RNFT fan W', 'profit': -13742, 'profit_p': -13.74, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 10/fan6:0/2 x l40 x 1.0¤ |u0.01 d0.0| W , 'op': 4205}
    {'exp': 'RNFT fan MW', 'profit': -14339, 'profit_p': -14.34, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 10/fan4:0/2 x l49 x 0.8¤ |u0.01 d0.0| MW , 'op': 3512}
    {'exp': 'RNFT fan ', 'profit': -14405, 'profit_p': -14.4, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 8/fan6:0/2 x l55 x 1.0¤ |u0.01 d0.0| , 'op': 4421}
    {'exp': 'RNFT fan E', 'profit': -14519, 'profit_p': -14.52, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 8/fan5:0/2 x l55 x 1.0¤ |u0.01 d0.0| E , 'op': 5947}
    {'exp': 'RNFT fan W', 'profit': -15190, 'profit_p': -15.19, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 10/fan5:0/2 x l39 x 1.0¤ |u0.01 d0.0| W , 'op': 4303}
    {'exp': 'RNFT fan ', 'profit': -15685, 'profit_p': -15.69, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 8/fan5:0/2 x l55 x 1.0¤ |u0.01 d0.0| , 'op': 4686}
    {'exp': 'RNFT fan EFMWB', 'profit': -16050, 'profit_p': -16.05, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': RNFT+ 8/fan3:0/2 x l48 x 1.0¤ |u0.01 d0.0| EFMWB , 'op': 10792}
    {'exp': 'RNFT fan MW', 'profit': -16292, 'profit_p': -16.29, 'profit_p_avg': -0.04, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 10/fan3:0/2 x l49 x 0.8¤ |u0.01 d0.0| MW , 'op': 4275}
    {'exp': 'RNFT fan W', 'profit': -16605, 'profit_p': -16.61, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 8/fan4:0/2 x l61 x 1.0¤ |u0.01 d0.0| W , 'op': 3850}
    {'exp': 'RNFT fan M', 'profit': -16756, 'profit_p': -16.76, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 10/fan4:0/2 x l44 x 0.8¤ |u0.01 d0.0| M , 'op': 3895}
    {'exp': 'RNFT fan E', 'profit': -17074, 'profit_p': -17.07, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 6/fan4:0/2 x l74 x 1.4¤ |u0.01 d0.0| E , 'op': 5510}
    {'exp': 'RNFT fan MW', 'profit': -17818, 'profit_p': -17.82, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': RNFT+ 6/fan5:0/2 x l63 x 1.8¤ |u0.01 d0.0| MW , 'op': 4115}
    {'exp': 'RNFT fan ', 'profit': -19036, 'profit_p': -19.04, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 13/fan4:0/2 x l34 x 0.6¤ |u0.01 d0.0| , 'op': 4387}
    {'exp': 'RNFT fan M', 'profit': -19273, 'profit_p': -19.27, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 10/fan5:0/2 x l44 x 0.8¤ |u0.01 d0.0| M , 'op': 4280}
    {'exp': 'RNFT fan W', 'profit': -19671, 'profit_p': -19.67, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': RNFT+ 8/fan3:0/2 x l61 x 1.0¤ |u0.01 d0.0| W , 'op': 5120}
    {'exp': 'RNFT fan M', 'profit': -20096, 'profit_p': -20.1, 'profit_p_avg': -0.05, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': RNFT+ 13/fan3:0/2 x l34 x 0.6¤ |u0.01 d0.0| M , 'op': 4622}
    {'exp': 'RNFT fan E', 'profit': -23087, 'profit_p': -23.09, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': RNFT+ 8/fan3:0/2 x l55 x 1.0¤ |u0.01 d0.0| E , 'op': 6058}
    {'exp': 'RNFT fan F', 'profit': -23667, 'profit_p': -23.67, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 8/fan7:0/2 x l39 x 1.0¤ |u0.01 d0.0| F , 'op': 9443}
    {'exp': 'RNFT fan ', 'profit': -23758, 'profit_p': -23.76, 'profit_p_avg': -0.06, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': RNFT+ 13/fan3:0/2 x l34 x 0.6¤ |u0.01 d0.0| , 'op': 5223}
    {'exp': 'RNFT fan F', 'profit': -27026, 'profit_p': -27.03, 'profit_p_avg': -0.07, 'config': RNFT+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 8/fan6:0/2 x l37 x 1.0¤ |u0.01 d0.0| F , 'op': 9539}
    {'exp': 'RNFT fan F', 'profit': -31531, 'profit_p': -31.53, 'profit_p_avg': -0.09, 'config': RNFT+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 8/fan5:0/2 x l35 x 1.0¤ |u0.01 d0.0| F , 'op': 9941}
    {'exp': 'RNFT fan F', 'profit': -31788, 'profit_p': -31.79, 'profit_p_avg': -0.09, 'config': RNFT+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 13/fan4:0/2 x l21 x 0.6¤ |u0.01 d0.0| F , 'op': 9668}
    {'exp': 'RNFT fan F', 'profit': -37411, 'profit_p': -37.41, 'profit_p_avg': -0.1, 'config': RNFT+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': RNFT+ 13/fan3:0/2 x l19 x 0.6¤ |u0.01 d0.0| F , 'op': 10982}

    {'exp': 'MVID fan E', 'profit': 3525, 'profit_p': 3.52, 'profit_p_avg': 0.01, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': MVID+ 14/fan4:0/2 x l39 x 0.4¤ |u0.01 d0.0| E , 'op': 3986}
    {'exp': 'MVID fan EB', 'profit': 1515, 'profit_p': 1.51, 'profit_p_avg': 0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': MVID+ 6/fan3:0/2 x l89 x 1.0¤ |u0.01 d0.0| EB , 'op': 2518}
    {'exp': 'MVID fan B', 'profit': 853, 'profit_p': 0.85, 'profit_p_avg': 0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': MVID+ 7/fan4:0/2 x l76 x 0.8¤ B , 'op': 2135}
    {'exp': 'MVID fan B', 'profit': 380, 'profit_p': 0.38, 'profit_p_avg': 0.0, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': MVID+ 6/fan6:0/2 x l88 x 1.0¤ B , 'op': 2282}
    {'exp': 'MVID fan E', 'profit': -420, 'profit_p': -0.42, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': MVID+ 14/fan5:0/2 x l37 x 0.4¤ |u0.01 d0.0| E , 'op': 3624}
    {'exp': 'MVID fan EMWB', 'profit': -917, 'profit_p': -0.92, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': MVID+ 13/fan5:0/2 x l43 x 1.0¤ EMWB , 'op': 1773}
    {'exp': 'MVID fan ', 'profit': -947, 'profit_p': -0.95, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': MVID+ 7/fan6:0/2 x l74 x 0.8¤ |u0.01 d0.0| , 'op': 3564}
    {'exp': 'MVID fan EB', 'profit': -1195, 'profit_p': -1.2, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': MVID+ 4/fan5:0/2 x l130 x 1.6(+x0.1)¤ EB , 'op': 2044}
    {'exp': 'MVID fan M', 'profit': -1387, 'profit_p': -1.39, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': MVID+ 6/fan4:0/2 x l87 x 1.0¤ |u0.01 d0.0| M , 'op': 3497}
    {'exp': 'MVID fan EB', 'profit': -1449, 'profit_p': -1.45, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': MVID+ 5/fan4:0/2 x l104 x 1.2¤ |u0.01 d0.0| EB , 'op': 2368}
    {'exp': 'MVID fan M', 'profit': -1670, 'profit_p': -1.67, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': MVID+ 6/fan5:0/2 x l86 x 1.0¤ |u0.01 d0.0| M , 'op': 3305}
    {'exp': 'MVID fan M', 'profit': -1744, 'profit_p': -1.74, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': MVID+ 6/fan6:0/2 x l86 x 1.0¤ |u0.01 d0.0| M , 'op': 3050}
    {'exp': 'MVID fan E', 'profit': -1786, 'profit_p': -1.79, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': MVID+ 14/fan3:0/2 x l37 x 0.4¤ |u0.01 d0.0| E , 'op': 4450}
    {'exp': 'MVID fan B', 'profit': -1870, 'profit_p': -1.87, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': MVID+ 7/fan6:0/2 x l74 x 0.8¤ |u0.01 d0.0| B , 'op': 2008}
    {'exp': 'MVID fan EB', 'profit': -1907, 'profit_p': -1.91, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': MVID+ 5/fan3:0/2 x l103 x 1.2¤ |u0.01 d0.0| EB , 'op': 2353}
    {'exp': 'MVID fan EFMWB', 'profit': -2708, 'profit_p': -2.71, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': MVID+ 11/fan5:0/2 x l50 x 1.2(+x0.1)¤ |u0.01 d0.0| EFMWB , 'op': 3085}
    {'exp': 'MVID fan B', 'profit': -3125, 'profit_p': -3.12, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': MVID+ 4/fan3:0/2 x l128 x 1.6(+x0.4)¤ |u0.01 d0.0| B , 'op': 1973}
    {'exp': 'MVID fan ', 'profit': -3204, 'profit_p': -3.2, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': MVID+ 7/fan5:0/2 x l73 x 0.8¤ |u0.01 d0.0| , 'op': 3920}
    {'exp': 'MVID fan EFMWB', 'profit': -3248, 'profit_p': -3.25, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': MVID+ 4/fan3:0/2 x l135 x 1.2¤ EFMWB , 'op': 4359}
    {'exp': 'MVID fan EMWB', 'profit': -3271, 'profit_p': -3.27, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': MVID+ 11/fan6:0/2 x l49 x 1.2¤ |u0.01 d0.0| EMWB , 'op': 1458}
    {'exp': 'MVID fan EFMWB', 'profit': -3480, 'profit_p': -3.48, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': MVID+ 11/fan5:0/2 x l49 x 1.2(+x0.1)¤ |u0.01 d0.0| EFMWB , 'op': 2778}
    {'exp': 'MVID fan EMWB', 'profit': -3717, 'profit_p': -3.72, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': MVID+ 4/fan4:0/2 x l135 x 1.2¤ EMWB , 'op': 2833}
    {'exp': 'MVID fan B', 'profit': -4340, 'profit_p': -4.34, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| B , 'last_conf': MVID+ 7/fan7:0/2 x l72 x 0.8¤ |u0.01 d0.0| B , 'op': 2307}
    {'exp': 'MVID fan M', 'profit': -4340, 'profit_p': -4.34, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': MVID+ 6/fan7:0/2 x l84 x 1.0¤ |u0.01 d0.0| M , 'op': 2825}
    {'exp': 'MVID fan EFMWB', 'profit': -4598, 'profit_p': -4.6, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': MVID+ 5/fan3:0/2 x l107 x 1.0¤ EFMWB , 'op': 4893}
    {'exp': 'MVID fan M', 'profit': -4808, 'profit_p': -4.81, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| M , 'last_conf': MVID+ 6/fan3:0/2 x l84 x 1.0¤ |u0.01 d0.0| M , 'op': 4155}
    {'exp': 'MVID fan EMWB', 'profit': -5228, 'profit_p': -5.23, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': MVID+ 13/fan5:0/2 x l40 x 1.0¤ EMWB , 'op': 2383}
    {'exp': 'MVID fan EB', 'profit': -5323, 'profit_p': -5.32, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| EB , 'last_conf': MVID+ 5/fan7:0/2 x l100 x 1.2¤ |u0.01 d0.0| EB , 'op': 2025}
    {'exp': 'MVID fan MW', 'profit': -5602, 'profit_p': -5.6, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': MVID+ 13/fan7:0/2 x l40 x 1.0¤ |u0.01 d0.0| MW , 'op': 2876}
    {'exp': 'MVID fan F', 'profit': -5615, 'profit_p': -5.62, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': MVID+ 7/fan6:0/2 x l71 x 0.8¤ |u0.01 d0.0| F , 'op': 5681}
    {'exp': 'MVID fan W', 'profit': -5628, 'profit_p': -5.63, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': MVID+ 13/fan6:0/2 x l40 x 1.0¤ |u0.01 d0.0| W , 'op': 3128}
    {'exp': 'MVID fan ', 'profit': -5727, 'profit_p': -5.73, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': MVID+ 6/fan4:0/2 x l83 x 1.0¤ |u0.01 d0.0| , 'op': 4150}
    {'exp': 'MVID fan MW', 'profit': -6097, 'profit_p': -6.1, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': MVID+ 13/fan6:0/2 x l40 x 1.0¤ |u0.01 d0.0| MW , 'op': 2827}
    {'exp': 'MVID fan MW', 'profit': -6381, 'profit_p': -6.38, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': MVID+ 13/fan5:0/2 x l40 x 1.0¤ |u0.01 d0.0| MW , 'op': 3030}
    {'exp': 'MVID fan EMWB', 'profit': -6440, 'profit_p': -6.44, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| EMWB , 'last_conf': MVID+ 5/fan3:0/2 x l105 x 1.0¤ |u0.01 d0.0| EMWB , 'op': 3351}
    {'exp': 'MVID fan F', 'profit': -6822, 'profit_p': -6.82, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': MVID+ 7/fan7:0/2 x l70 x 0.8¤ |u0.01 d0.0| F , 'op': 5588}
    {'exp': 'MVID fan EFMWB', 'profit': -7762, 'profit_p': -7.76, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| EFMWB , 'last_conf': MVID+ 11/fan7:0/2 x l47 x 1.2¤ EFMWB , 'op': 6334}
    {'exp': 'MVID fan MW', 'profit': -8150, 'profit_p': -8.15, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': MVID+ 5/fan4:0/2 x l103 x 1.0¤ |u0.01 d0.0| MW , 'op': 3246}
    {'exp': 'MVID fan E', 'profit': -8464, 'profit_p': -8.46, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan6:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': MVID+ 6/fan6:0/2 x l80 x 1.0¤ |u0.01 d0.0| E , 'op': 3412}
    {'exp': 'MVID fan W', 'profit': -8840, 'profit_p': -8.84, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': MVID+ 13/fan5:0/2 x l39 x 1.0¤ |u0.01 d0.0| W , 'op': 3473}
    {'exp': 'MVID fan ', 'profit': -8912, 'profit_p': -8.91, 'profit_p_avg': -0.02, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': MVID+ 7/fan3:0/2 x l68 x 0.8¤ |u0.01 d0.0| , 'op': 4498}
    {'exp': 'MVID fan F', 'profit': -9755, 'profit_p': -9.76, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan5:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': MVID+ 7/fan5:0/2 x l68 x 0.8¤ |u0.01 d0.0| F , 'op': 5905}
    {'exp': 'MVID fan MW', 'profit': -10086, 'profit_p': -10.09, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| MW , 'last_conf': MVID+ 5/fan3:0/2 x l100 x 1.0¤ |u0.01 d0.0| MW , 'op': 3747}
    {'exp': 'MVID fan F', 'profit': -10811, 'profit_p': -10.81, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': MVID+ 7/fan3:0/2 x l67 x 0.8¤ |u0.01 d0.0| F , 'op': 6771}
    {'exp': 'MVID fan E', 'profit': -10908, 'profit_p': -10.91, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| E , 'last_conf': MVID+ 5/fan7:0/2 x l94 x 1.2¤ |u0.01 d0.0| E , 'op': 3365}
    {'exp': 'MVID fan W', 'profit': -10926, 'profit_p': -10.93, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': MVID+ 6/fan4:0/2 x l83 x 0.8¤ |u0.01 d0.0| W , 'op': 3594}
    {'exp': 'MVID fan F', 'profit': -11183, 'profit_p': -11.18, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan4:0/2 x l1 x 0.8¤ |u0.01 d0.0| F , 'last_conf': MVID+ 6/fan4:0/2 x l78 x 1.0¤ |u0.01 d0.0| F , 'op': 6193}
    {'exp': 'MVID fan ', 'profit': -11405, 'profit_p': -11.4, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| , 'last_conf': MVID+ 7/fan7:0/2 x l67 x 0.8¤ |u0.01 d0.0| , 'op': 3525}
    {'exp': 'MVID fan W', 'profit': -11592, 'profit_p': -11.59, 'profit_p_avg': -0.03, 'config': MVID+ 4/fan7:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': MVID+ 13/fan7:0/2 x l38 x 1.0¤ |u0.01 d0.0| W , 'op': 3256}
    {'exp': 'MVID fan W', 'profit': -13007, 'profit_p': -13.01, 'profit_p_avg': -0.04, 'config': MVID+ 4/fan3:0/2 x l1 x 0.8¤ |u0.01 d0.0| W , 'last_conf': MVID+ 5/fan3:0/2 x l97 x 1.0¤ |u0.01 d0.0| W , 'op': 3630}

    {'exp': 'MTLR pre B', 'profit': 7552, 'profit_p': 7.55, 'profit_p_avg': 0.02, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ B , 'last_conf': MTLR+ 23/pre5:23/2 x l21 x 3.2(+x0.2)¤ B , 'op': 3780}
    {'exp': 'MTLR pre EFMWB', 'profit': 3199, 'profit_p': 3.2, 'profit_p_avg': 0.01, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ EFMWB , 'last_conf': MTLR+ 8/pre6:8/2 x l67 x 0.8(+x0.1)¤ |u0.01 d0.0| EFMWB , 'op': 3182}
    {'exp': 'MTLR pre MW', 'profit': 2745, 'profit_p': 2.75, 'profit_p_avg': 0.01, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ MW , 'last_conf': MTLR+ 8/pre7:8/2 x l67 x 1.6¤ MW , 'op': 5220}
    {'exp': 'MTLR pre EFMWB', 'profit': 2216, 'profit_p': 2.22, 'profit_p_avg': 0.01, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ EFMWB , 'last_conf': MTLR+ 7/pre6:7/2 x l77 x 1.6(+x0.2)¤ EFMWB , 'op': 3059}
    {'exp': 'MTLR pre EB', 'profit': 2008, 'profit_p': 2.01, 'profit_p_avg': 0.01, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ EB , 'last_conf': MTLR+ 24/pre6:24/2 x l19 x 1.2(+x0.2)¤ EB , 'op': 10562}
    {'exp': 'MTLR pre EMWB', 'profit': 705, 'profit_p': 0.7, 'profit_p_avg': 0.0, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ EMWB , 'last_conf': MTLR+ 7/pre6:7/2 x l75 x 1.6(+x0.2)¤ EMWB , 'op': 2748}
    {'exp': 'MTLR pre EMWB', 'profit': -895, 'profit_p': -0.9, 'profit_p_avg': -0.0, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ EMWB , 'last_conf': MTLR+ 11/pre4:11/2 x l47 x 1.6(+x0.3)¤ EMWB , 'op': 4091}
    {'exp': 'MTLR pre W', 'profit': -2734, 'profit_p': -2.73, 'profit_p_avg': -0.01, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ W , 'last_conf': MTLR+ 10/pre7:10/2 x l51 x 1.4¤ W , 'op': 6297}
    {'exp': 'MTLR pre EMWB', 'profit': -5906, 'profit_p': -5.91, 'profit_p_avg': -0.02, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ EMWB , 'last_conf': MTLR+ 8/pre5:8/2 x l61 x 0.8¤ |u0.01 d0.0| EMWB , 'op': 2638}
    {'exp': 'MTLR pre EFMWB', 'profit': -6007, 'profit_p': -6.01, 'profit_p_avg': -0.02, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ EFMWB , 'last_conf': MTLR+ 11/pre4:11/2 x l45 x 1.6(+x0.1)¤ EFMWB , 'op': 4404}
    {'exp': 'MTLR pre M', 'profit': -7334, 'profit_p': -7.33, 'profit_p_avg': -0.02, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ M , 'last_conf': MTLR+ 9/pre7:9/2 x l47 x 1.8¤ M , 'op': 6378}
    {'exp': 'MTLR pre MW', 'profit': -9139, 'profit_p': -9.14, 'profit_p_avg': -0.02, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ MW , 'last_conf': MTLR+ 7/pre6:7/2 x l68 x 1.4¤ MW , 'op': 5576}
    {'exp': 'MTLR pre B', 'profit': -9868, 'profit_p': -9.87, 'profit_p_avg': -0.03, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ B , 'last_conf': MTLR+ 15/pre4:15/2 x l27 x 2.4(+x0.2)¤ B , 'op': 3554}
    {'exp': 'MTLR pre EMWB', 'profit': -9903, 'profit_p': -9.9, 'profit_p_avg': -0.03, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ EMWB , 'last_conf': MTLR+ 9/pre3:9/2 x l52 x 1.8¤ EMWB , 'op': 2442}
    {'exp': 'MTLR pre EB', 'profit': -10574, 'profit_p': -10.57, 'profit_p_avg': -0.03, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ EB , 'last_conf': MTLR+ 15/pre5:15/2 x l27 x 2.4¤ EB , 'op': 3542}
    {'exp': 'MTLR pre W', 'profit': -11548, 'profit_p': -11.55, 'profit_p_avg': -0.03, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ W , 'last_conf': MTLR+ 11/pre6:11/2 x l42 x 1.4¤ W , 'op': 6655}
    {'exp': 'MTLR pre EFMWB', 'profit': -12279, 'profit_p': -12.28, 'profit_p_avg': -0.03, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ EFMWB , 'last_conf': MTLR+ 12/pre4:12/2 x l38 x 1.6(+x0.4)¤ EFMWB , 'op': 3673}
    {'exp': 'MTLR pre M', 'profit': -13514, 'profit_p': -13.51, 'profit_p_avg': -0.04, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ M , 'last_conf': MTLR+ 9/pre6:9/2 x l44 x 2.0¤ M , 'op': 6224}
    {'exp': 'MTLR pre EMWB', 'profit': -13622, 'profit_p': -13.62, 'profit_p_avg': -0.04, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ EMWB , 'last_conf': MTLR+ 12/pre3:12/2 x l38 x 1.6(+x0.4)¤ EMWB , 'op': 3166}
    {'exp': 'MTLR pre W', 'profit': -14124, 'profit_p': -14.12, 'profit_p_avg': -0.04, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ W , 'last_conf': MTLR+ 6/pre3:6/2 x l75 x 1.0¤ W , 'op': 5455}
    {'exp': 'MTLR pre B', 'profit': -17010, 'profit_p': -17.01, 'profit_p_avg': -0.05, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ B , 'last_conf': MTLR+ 9/pre6:9/2 x l42 x 1.2¤ B , 'op': 4250}
    {'exp': 'MTLR pre MW', 'profit': -17473, 'profit_p': -17.47, 'profit_p_avg': -0.05, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ MW , 'last_conf': MTLR+ 5/pre3:5/2 x l110 x 0.6¤ MW , 'op': 4879}
    {'exp': 'MTLR pre EB', 'profit': -19030, 'profit_p': -19.03, 'profit_p_avg': -0.05, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ EB , 'last_conf': MTLR+ 9/pre4:9/2 x l41 x 1.2(+x0.1)¤ EB , 'op': 4222}
    {'exp': 'MTLR pre EFMWB', 'profit': -19164, 'profit_p': -19.16, 'profit_p_avg': -0.05, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ EFMWB , 'last_conf': MTLR+ 16/pre6:16/2 x l26 x 1.6(+x0.1)¤ EFMWB , 'op': 3511}
    {'exp': 'MTLR pre F', 'profit': -19784, 'profit_p': -19.78, 'profit_p_avg': -0.05, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ F , 'last_conf': MTLR+ 5/pre7:5/2 x l73 x 0.6¤ F , 'op': 8509}
    {'exp': 'MTLR pre M', 'profit': -26332, 'profit_p': -26.33, 'profit_p_avg': -0.07, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ M , 'last_conf': MTLR+ 6/pre3:6/2 x l57 x 0.8¤ M , 'op': 6240}
    {'exp': 'MTLR pre ', 'profit': -26562, 'profit_p': -26.56, 'profit_p_avg': -0.07, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ , 'last_conf': MTLR+ 5/pre7:5/2 x l103 x 0.6¤ , 'op': 6695}
    {'exp': 'MTLR pre M', 'profit': -26725, 'profit_p': -26.73, 'profit_p_avg': -0.07, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ M , 'last_conf': MTLR+ 11/pre5:11/2 x l30 x 1.0¤ M , 'op': 5865}
    {'exp': 'MTLR pre E', 'profit': -26856, 'profit_p': -26.86, 'profit_p_avg': -0.07, 'config': MTLR+ 4/pre7:0/2 x l1 x 0.4¤ E , 'last_conf': MTLR+ 5/pre7:5/2 x l103 x 0.6¤ E , 'op': 7271}
    {'exp': 'MTLR pre EB', 'profit': -27291, 'profit_p': -27.29, 'profit_p_avg': -0.07, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ EB , 'last_conf': MTLR+ 8/pre3:8/2 x l41 x 3.2(+x0.2)¤ EB , 'op': 2502}
    {'exp': 'MTLR pre EB', 'profit': -27628, 'profit_p': -27.63, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ EB , 'last_conf': MTLR+ 12/pre4:12/2 x l27 x 2.6(+x0.1)¤ EB , 'op': 2993}
    {'exp': 'MTLR pre F', 'profit': -28127, 'profit_p': -28.13, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ F , 'last_conf': MTLR+ 5/pre6:5/2 x l66 x 0.6¤ F , 'op': 8942}
    {'exp': 'MTLR pre MW', 'profit': -28652, 'profit_p': -28.65, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ MW , 'last_conf': MTLR+ 5/pre4:5/2 x l74 x 1.0¤ MW , 'op': 4751}
    {'exp': 'MTLR pre ', 'profit': -28739, 'profit_p': -28.74, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ , 'last_conf': MTLR+ 5/pre6:5/2 x l104 x 0.6¤ , 'op': 8038}
    {'exp': 'MTLR pre MW', 'profit': -28800, 'profit_p': -28.8, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ MW , 'last_conf': MTLR+ 5/pre5:5/2 x l74 x 1.4¤ MW , 'op': 4867}
    {'exp': 'MTLR pre B', 'profit': -28965, 'profit_p': -28.96, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ B , 'last_conf': MTLR+ 12/pre4:12/2 x l27 x 2.6(+x0.2)¤ B , 'op': 2986}
    {'exp': 'MTLR pre M', 'profit': -29792, 'profit_p': -29.79, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ M , 'last_conf': MTLR+ 5/pre4:5/2 x l65 x 0.6¤ M , 'op': 6364}
    {'exp': 'MTLR pre E', 'profit': -30918, 'profit_p': -30.92, 'profit_p_avg': -0.08, 'config': MTLR+ 4/pre6:0/2 x l1 x 0.4¤ E , 'last_conf': MTLR+ 5/pre6:5/2 x l104 x 0.6¤ E , 'op': 7818}
    {'exp': 'MTLR pre W', 'profit': -31953, 'profit_p': -31.95, 'profit_p_avg': -0.09, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ W , 'last_conf': MTLR+ 5/pre5:5/2 x l71 x 0.8¤ W , 'op': 5368}
    {'exp': 'MTLR pre W', 'profit': -31985, 'profit_p': -31.98, 'profit_p_avg': -0.09, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ W , 'last_conf': MTLR+ 4/pre4:4/2 x l138 x 0.4¤ W , 'op': 5436}
    {'exp': 'MTLR pre F', 'profit': -35331, 'profit_p': -35.33, 'profit_p_avg': -0.1, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ F , 'last_conf': MTLR+ 7/pre3:7/2 x l42 x 0.8¤ F , 'op': 7894}
    {'exp': 'MTLR pre ', 'profit': -36839, 'profit_p': -36.84, 'profit_p_avg': -0.1, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ , 'last_conf': MTLR+ 7/pre3:7/2 x l75 x 0.8¤ , 'op': 6533}
    {'exp': 'MTLR pre E', 'profit': -36955, 'profit_p': -36.95, 'profit_p_avg': -0.1, 'config': MTLR+ 4/pre3:0/2 x l1 x 0.4¤ E , 'last_conf': MTLR+ 7/pre3:7/2 x l75 x 0.8¤ E , 'op': 6466}
    {'exp': 'MTLR pre B', 'profit': -37847, 'profit_p': -37.85, 'profit_p_avg': -0.1, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ B , 'last_conf': MTLR+ 11/pre6:11/2 x l26 x 2.2(+x0.3)¤ B , 'op': 3446}
    {'exp': 'MTLR pre F', 'profit': -39012, 'profit_p': -39.01, 'profit_p_avg': -0.11, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ F , 'last_conf': MTLR+ 10/pre4:10/2 x l28 x 0.6¤ F , 'op': 9298}
    {'exp': 'MTLR pre E', 'profit': -39695, 'profit_p': -39.7, 'profit_p_avg': -0.11, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ E , 'last_conf': MTLR+ 6/pre5:6/2 x l89 x 0.4¤ E , 'op': 7556}
    {'exp': 'MTLR pre F', 'profit': -39913, 'profit_p': -39.91, 'profit_p_avg': -0.11, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ F , 'last_conf': MTLR+ 6/pre5:6/2 x l45 x 0.4¤ F , 'op': 10355}
    {'exp': 'MTLR pre ', 'profit': -40582, 'profit_p': -40.58, 'profit_p_avg': -0.11, 'config': MTLR+ 4/pre5:0/2 x l1 x 0.4¤ , 'last_conf': MTLR+ 6/pre5:6/2 x l89 x 0.4¤ , 'op': 7506}
    {'exp': 'MTLR pre ', 'profit': -41852, 'profit_p': -41.85, 'profit_p_avg': -0.11, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ , 'last_conf': MTLR+ 10/pre4:10/2 x l53 x 0.6¤ , 'op': 7569}
    {'exp': 'MTLR pre E', 'profit': -42051, 'profit_p': -42.05, 'profit_p_avg': -0.11, 'config': MTLR+ 4/pre4:0/2 x l1 x 0.4¤ E , 'last_conf': MTLR+ 10/pre4:10/2 x l53 x 0.6¤ E , 'op': 7542}

    {'exp': 'MTLR fan EMWB', 'profit': -5151, 'profit_p': -5.15, 'profit_p_avg': -0.01, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| EMWB , 'last_conf': MTLR+ 4/fan3:0/2 x l138 x 1.8(+x0.2)¤ |u0.01 d0.0| EMWB , 'op': 1033}
    {'exp': 'MTLR fan W', 'profit': -7614, 'profit_p': -7.61, 'profit_p_avg': -0.02, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| W , 'last_conf': MTLR+ 11/fan4:0/2 x l50 x 0.4¤ |u0.01 d0.0| W , 'op': 3531}
    {'exp': 'MTLR fan EMWB', 'profit': -8062, 'profit_p': -8.06, 'profit_p_avg': -0.02, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| EMWB , 'last_conf': MTLR+ 4/fan3:0/2 x l138 x 1.2(+x0.1)¤ EMWB , 'op': 1291}
    {'exp': 'MTLR fan EMWB', 'profit': -8241, 'profit_p': -8.24, 'profit_p_avg': -0.02, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| EMWB , 'last_conf': MTLR+ 4/fan7:0/2 x l120 x 1.2(+x0.2)¤ |u0.01 d0.0| EMWB , 'op': 763}
    {'exp': 'MTLR fan EB', 'profit': -9526, 'profit_p': -9.53, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| EB , 'last_conf': MTLR+ 9/fan4:0/2 x l58 x 1.6(+x0.2)¤ |u0.01 d0.0| EB , 'op': 1192}
    {'exp': 'MTLR fan EMWB', 'profit': -10551, 'profit_p': -10.55, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| EMWB , 'last_conf': MTLR+ 4/fan5:0/2 x l117 x 1.2(+x0.2)¤ |u0.01 d0.0| EMWB , 'op': 787}
    {'exp': 'MTLR fan B', 'profit': -10629, 'profit_p': -10.63, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| B , 'last_conf': MTLR+ 9/fan3:0/2 x l58 x 1.6¤ B , 'op': 1079}
    {'exp': 'MTLR fan EMWB', 'profit': -10769, 'profit_p': -10.77, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| EMWB , 'last_conf': MTLR+ 4/fan3:0/2 x l117 x 1.2(+x0.2)¤ |u0.01 d0.0| EMWB , 'op': 745}
    {'exp': 'MTLR fan EFMWB', 'profit': -10791, 'profit_p': -10.79, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| EFMWB , 'last_conf': MTLR+ 4/fan5:0/2 x l117 x 1.2(+x0.1)¤ EFMWB , 'op': 4129}
    {'exp': 'MTLR fan W', 'profit': -11676, 'profit_p': -11.68, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| W , 'last_conf': MTLR+ 6/fan6:0/2 x l77 x 0.8¤ |u0.01 d0.0| W , 'op': 3218}
    {'exp': 'MTLR fan EFMWB', 'profit': -11747, 'profit_p': -11.75, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| EFMWB , 'last_conf': MTLR+ 4/fan5:0/2 x l138 x 1.2(+x0.2)¤ |u0.01 d0.0| EFMWB , 'op': 3937}
    {'exp': 'MTLR fan EB', 'profit': -12168, 'profit_p': -12.17, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| EB , 'last_conf': MTLR+ 7/fan3:0/2 x l75 x 2.2¤ EB , 'op': 855}
    {'exp': 'MTLR fan EFMWB', 'profit': -12294, 'profit_p': -12.29, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| EFMWB , 'last_conf': MTLR+ 4/fan5:0/2 x l115 x 1.2(+x0.2)¤ |u0.01 d0.0| EFMWB , 'op': 3885}
    {'exp': 'MTLR fan W', 'profit': -12484, 'profit_p': -12.48, 'profit_p_avg': -0.03, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| W , 'last_conf': MTLR+ 6/fan5:0/2 x l76 x 0.8¤ |u0.01 d0.0| W , 'op': 3723}
    {'exp': 'MTLR fan EFMWB', 'profit': -12810, 'profit_p': -12.81, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| EFMWB , 'last_conf': MTLR+ 4/fan4:0/2 x l114 x 1.8(+x0.2)¤ |u0.01 d0.0| EFMWB , 'op': 3751}
    {'exp': 'MTLR fan W', 'profit': -12837, 'profit_p': -12.84, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| W , 'last_conf': MTLR+ 6/fan7:0/2 x l76 x 0.8¤ |u0.01 d0.0| W , 'op': 2925}
    {'exp': 'MTLR fan EB', 'profit': -13121, 'profit_p': -13.12, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| EB , 'last_conf': MTLR+ 10/fan3:0/2 x l52 x 1.4(+x0.3)¤ |u0.01 d0.0| EB , 'op': 808}
    {'exp': 'MTLR fan EFMWB', 'profit': -13513, 'profit_p': -13.51, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| EFMWB , 'last_conf': MTLR+ 4/fan4:0/2 x l114 x 1.2(+x0.2)¤ EFMWB , 'op': 4107}
    {'exp': 'MTLR fan B', 'profit': -13819, 'profit_p': -13.82, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| B , 'last_conf': MTLR+ 7/fan4:0/2 x l75 x 2.2(+x0.2)¤ |u0.01 d0.0| B , 'op': 762}
    {'exp': 'MTLR fan EB', 'profit': -13980, 'profit_p': -13.98, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| EB , 'last_conf': MTLR+ 7/fan5:0/2 x l75 x 2.2(+x0.2)¤ |u0.01 d0.0| EB , 'op': 991}
    {'exp': 'MTLR fan B', 'profit': -14413, 'profit_p': -14.41, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| B , 'last_conf': MTLR+ 7/fan7:0/2 x l75 x 2.2(+x0.2)¤ |u0.01 d0.0| B , 'op': 977}
    {'exp': 'MTLR fan E', 'profit': -15009, 'profit_p': -15.01, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| E , 'last_conf': MTLR+ 23/fan7:0/2 x l22 x 0.6¤ |u0.01 d0.0| E , 'op': 3212}
    {'exp': 'MTLR fan MW', 'profit': -15367, 'profit_p': -15.37, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| MW , 'last_conf': MTLR+ 8/fan7:0/2 x l55 x 0.6¤ |u0.01 d0.0| MW , 'op': 2278}
    {'exp': 'MTLR fan ', 'profit': -15410, 'profit_p': -15.41, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 23/fan6:0/2 x l22 x 0.6¤ |u0.01 d0.0| , 'op': 4182}
    {'exp': 'MTLR fan EB', 'profit': -15751, 'profit_p': -15.75, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| EB , 'last_conf': MTLR+ 12/fan3:0/2 x l43 x 1.2(+x0.3)¤ |u0.01 d0.0| EB , 'op': 1097}
    {'exp': 'MTLR fan ', 'profit': -16126, 'profit_p': -16.13, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 23/fan5:0/2 x l22 x 0.6¤ |u0.01 d0.0| , 'op': 4440}
    {'exp': 'MTLR fan B', 'profit': -16410, 'profit_p': -16.41, 'profit_p_avg': -0.04, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| B , 'last_conf': MTLR+ 9/fan5:0/2 x l58 x 1.6(+x0.4)¤ |u0.01 d0.0| B , 'op': 1203}
    {'exp': 'MTLR fan ', 'profit': -17874, 'profit_p': -17.87, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 23/fan7:0/2 x l22 x 0.6¤ |u0.01 d0.0| , 'op': 3912}
    {'exp': 'MTLR fan MW', 'profit': -18198, 'profit_p': -18.2, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| MW , 'last_conf': MTLR+ 8/fan6:0/2 x l53 x 0.6¤ |u0.01 d0.0| MW , 'op': 2516}
    {'exp': 'MTLR fan E', 'profit': -18752, 'profit_p': -18.75, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| E , 'last_conf': MTLR+ 23/fan6:0/2 x l22 x 0.6¤ |u0.01 d0.0| E , 'op': 3897}
    {'exp': 'MTLR fan MW', 'profit': -18807, 'profit_p': -18.81, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| MW , 'last_conf': MTLR+ 8/fan5:0/2 x l53 x 0.6¤ |u0.01 d0.0| MW , 'op': 2842}
    {'exp': 'MTLR fan B', 'profit': -19034, 'profit_p': -19.03, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| B , 'last_conf': MTLR+ 23/fan7:0/2 x l22 x 0.6¤ |u0.01 d0.0| B , 'op': 3710}
    {'exp': 'MTLR fan MW', 'profit': -19218, 'profit_p': -19.22, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| MW , 'last_conf': MTLR+ 11/fan3:0/2 x l50 x 0.4¤ |u0.01 d0.0| MW , 'op': 3097}
    {'exp': 'MTLR fan M', 'profit': -19232, 'profit_p': -19.23, 'profit_p_avg': -0.05, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| M , 'last_conf': MTLR+ 34/fan7:0/2 x l15 x 0.4¤ |u0.01 d0.0| M , 'op': 2762}
    {'exp': 'MTLR fan MW', 'profit': -20156, 'profit_p': -20.16, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| MW , 'last_conf': MTLR+ 11/fan4:0/2 x l50 x 0.4¤ |u0.01 d0.0| MW , 'op': 2954}
    {'exp': 'MTLR fan W', 'profit': -20294, 'profit_p': -20.29, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| W , 'last_conf': MTLR+ 11/fan3:0/2 x l38 x 0.4¤ |u0.01 d0.0| W , 'op': 3949}
    {'exp': 'MTLR fan ', 'profit': -20375, 'profit_p': -20.38, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 23/fan4:0/2 x l22 x 0.6¤ |u0.01 d0.0| , 'op': 4862}
    {'exp': 'MTLR fan M', 'profit': -20685, 'profit_p': -20.68, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| M , 'last_conf': MTLR+ 34/fan6:0/2 x l15 x 0.4¤ |u0.01 d0.0| M , 'op': 2968}
    {'exp': 'MTLR fan E', 'profit': -21395, 'profit_p': -21.39, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| E , 'last_conf': MTLR+ 23/fan5:0/2 x l22 x 0.6¤ |u0.01 d0.0| E , 'op': 4097}
    {'exp': 'MTLR fan M', 'profit': -21699, 'profit_p': -21.7, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| M , 'last_conf': MTLR+ 23/fan5:0/2 x l22 x 0.6¤ |u0.01 d0.0| M , 'op': 3315}
    {'exp': 'MTLR fan E', 'profit': -22191, 'profit_p': -22.19, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| E , 'last_conf': MTLR+ 23/fan3:0/2 x l22 x 0.6¤ |u0.01 d0.0| E , 'op': 4506}
    {'exp': 'MTLR fan E', 'profit': -22306, 'profit_p': -22.31, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| E , 'last_conf': MTLR+ 23/fan4:0/2 x l22 x 0.6¤ |u0.01 d0.0| E , 'op': 3928}
    {'exp': 'MTLR fan M', 'profit': -23001, 'profit_p': -23.0, 'profit_p_avg': -0.06, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| M , 'last_conf': MTLR+ 34/fan4:0/2 x l15 x 0.4¤ |u0.01 d0.0| M , 'op': 4284}
    {'exp': 'MTLR fan M', 'profit': -25271, 'profit_p': -25.27, 'profit_p_avg': -0.07, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| M , 'last_conf': MTLR+ 23/fan3:0/2 x l22 x 0.6¤ |u0.01 d0.0| M , 'op': 4930}
    {'exp': 'MTLR fan ', 'profit': -27129, 'profit_p': -27.13, 'profit_p_avg': -0.07, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 23/fan3:0/2 x l22 x 0.6¤ |u0.01 d0.0| , 'op': 5709}
    {'exp': 'MTLR fan F', 'profit': -28425, 'profit_p': -28.43, 'profit_p_avg': -0.08, 'config': MTLR+ 4/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 23/fan6:0/2 x l14 x 0.6¤ |u0.01 d0.0| F , 'op': 8812}
    {'exp': 'MTLR fan F', 'profit': -29094, 'profit_p': -29.09, 'profit_p_avg': -0.08, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 23/fan7:0/2 x l14 x 0.6¤ |u0.01 d0.0| F , 'op': 9183}
    {'exp': 'MTLR fan F', 'profit': -30863, 'profit_p': -30.86, 'profit_p_avg': -0.08, 'config': MTLR+ 4/fan5:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 23/fan5:0/2 x l13 x 0.6¤ |u0.01 d0.0| F , 'op': 9748}
    {'exp': 'MTLR fan F', 'profit': -33738, 'profit_p': -33.74, 'profit_p_avg': -0.09, 'config': MTLR+ 4/fan4:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 23/fan4:0/2 x l13 x 0.6¤ |u0.01 d0.0| F , 'op': 11261}
    {'exp': 'MTLR fan F', 'profit': -34642, 'profit_p': -34.64, 'profit_p_avg': -0.09, 'config': MTLR+ 4/fan3:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 23/fan3:0/2 x l13 x 0.6¤ |u0.01 d0.0| F , 'op': 11412}